In [4]:
import json

def parse_paragraphs(data, paragraph_gap=20):
    fields = data["fields"]
    # 1) y 기준으로 정렬
    fields.sort(key=lambda f: f["boundingPoly"]["vertices"][0]["y"])
    
    # 2) 같은 줄(line) 묶기: lineBreak가 True일 때마다 새 줄
    lines = []
    cur_line = []
    for f in fields:
        text = f["inferText"]
        y = f["boundingPoly"]["vertices"][0]["y"]
        cur_line.append((text, f["boundingPoly"]["vertices"][0]))
        if f.get("lineBreak", False):
            lines.append(cur_line)
            cur_line = []
    if cur_line:
        lines.append(cur_line)
    
    # 3) 단락 묶기: 줄 간 y 차이가 paragraph_gap 이상일 때 새 단락
    paragraphs = []
    cur_para = []
    prev_y = None
    for line in lines:
        # 이 줄의 최소 y
        line_y = min(pt["y"] for _, pt in line)
        if prev_y is not None and line_y - prev_y > paragraph_gap:
            # 새로운 단락 시작
            paragraphs.append(cur_para)
            cur_para = []
        cur_para.append(line)
        prev_y = line_y
    if cur_para:
        paragraphs.append(cur_para)
    
    # 4) 결과 포맷: 각 단락에 (text, top_left)
    result = []
    for para in paragraphs:
        # 단락 전체 텍스트: 줄별로 합치고 줄 바꿈
        texts = [" ".join(word for word, _ in line) for line in para]
        full_text = "\n".join(texts)
        # 단락의 top-left: 모든 단어 중 min x, min y
        all_pts = [pt for line in para for _, pt in line]
        top_left = {"x": min(p["x"] for p in all_pts), "y": min(p["y"] for p in all_pts)}
        result.append({"text": full_text, "top_left": top_left})
    
    return result



In [ ]:

# 사용 예시
# with open("입력.json", "r", encoding="utf-8") as f:
#     data = json.load(f)

paras = parse_paragraphs(data)
for i, p in enumerate(paras, 1):
    print(f"[단락 {i}] 좌표={p['top_left']}:\n{p['text']}\n")

In [ ]:
import json
raw_json = """{"version":"V2","requestId":"888b14d2addd4ac0aa7bfe693789d044","timestamp":1747200298653,"images":[{"uid":"e5144fccfdee49cdb3918da78c305ff6","name":"demo","inferResult":"SUCCESS","message":"SUCCESS","validationResult":{"result":"NO_REQUESTED"},"convertedImageInfo":{"width":750,"height":5216,"pageIndex":0,"longImage":false},"tables":[{"cells":[{"cellTextLines":[{"boundingPoly":{"vertices":[{"x":228.0,"y":1830.0},{"x":526.0,"y":1830.0},{"x":526.0,"y":1868.0},{"x":228.0,"y":1868.0}]},"inferConfidence":0.99925,"cellWords":[{"boundingPoly":{"vertices":[{"x":228.0,"y":1830.0},{"x":405.0,"y":1830.0},{"x":405.0,"y":1868.0},{"x":228.0,"y":1868.0}]},"inferText":"본인 확인 가능한","inferConfidence":0.9988},{"boundingPoly":{"vertices":[{"x":396.0,"y":1830.0},{"x":526.0,"y":1830.0},{"x":526.0,"y":1868.0},{"x":396.0,"y":1868.0}]},"inferText":"유효 신분증","inferConfidence":0.9997}]}],"boundingPoly":{"vertices":[{"x":42.0,"y":1819.0},{"x":713.0,"y":1819.0},{"x":713.0,"y":1877.0},{"x":42.0,"y":1877.0}]},"inferConfidence":0.99925,"rowSpan":1,"rowIndex":0,"columnSpan":1,"columnIndex":0},{"cellTextLines":[{"boundingPoly":{"vertices":[{"x":270.0,"y":1914.0},{"x":484.0,"y":1914.0},{"x":484.0,"y":1951.0},{"x":270.0,"y":1951.0}]},"inferConfidence":0.99975,"cellWords":[{"boundingPoly":{"vertices":[{"x":270.0,"y":1914.0},{"x":391.0,"y":1914.0},{"x":391.0,"y":1951.0},{"x":270.0,"y":1951.0}]},"inferText":"주민등록증","inferConfidence":0.9998},{"boundingPoly":{"vertices":[{"x":382.0,"y":1914.0},{"x":484.0,"y":1914.0},{"x":484.0,"y":1951.0},{"x":382.0,"y":1951.0}]},"inferText":"기발급자","inferConfidence":0.9997}]},{"boundingPoly":{"vertices":[{"x":154.0,"y":1970.0},{"x":596.0,"y":1970.0},{"x":596.0,"y":2017.0},{"x":154.0,"y":2017.0}]},"inferConfidence":0.99418,"cellWords":[{"boundingPoly":{"vertices":[{"x":154.0,"y":1970.0},{"x":284.0,"y":1970.0},{"x":284.0,"y":2017.0},{"x":154.0,"y":2017.0}]},"inferText":"주민등록증,","inferConfidence":0.9999},{"boundingPoly":{"vertices":[{"x":270.0,"y":1970.0},{"x":396.0,"y":1970.0},{"x":396.0,"y":2017.0},{"x":270.0,"y":2017.0}]},"inferText":", 운전면허증,","inferConfidence":0.9739},{"boundingPoly":{"vertices":[{"x":391.0,"y":1975.0},{"x":494.0,"y":1975.0},{"x":494.0,"y":2012.0},{"x":391.0,"y":2012.0}]},"inferText":"기간만료","inferConfidence":0.9974},{"boundingPoly":{"vertices":[{"x":484.0,"y":1975.0},{"x":540.0,"y":1975.0},{"x":540.0,"y":2012.0},{"x":484.0,"y":2012.0}]},"inferText":"전의","inferConfidence":0.9997},{"boundingPoly":{"vertices":[{"x":536.0,"y":1975.0},{"x":596.0,"y":1975.0},{"x":596.0,"y":2017.0},{"x":536.0,"y":2017.0}]},"inferText":"여권,","inferConfidence":1.0}]},{"boundingPoly":{"vertices":[{"x":163.0,"y":2012.0},{"x":587.0,"y":2012.0},{"x":587.0,"y":2054.0},{"x":163.0,"y":2054.0}]},"inferConfidence":0.999425,"cellWords":[{"boundingPoly":{"vertices":[{"x":163.0,"y":2017.0},{"x":284.0,"y":2017.0},{"x":284.0,"y":2054.0},{"x":163.0,"y":2054.0}]},"inferText":"주민등록증","inferConfidence":0.9999},{"boundingPoly":{"vertices":[{"x":275.0,"y":2012.0},{"x":442.0,"y":2012.0},{"x":442.0,"y":2054.0},{"x":275.0,"y":2054.0}]},"inferText":"발급신청확인서,","inferConfidence":0.9978},{"boundingPoly":{"vertices":[{"x":442.0,"y":2017.0},{"x":517.0,"y":2017.0},{"x":517.0,"y":2054.0},{"x":442.0,"y":2054.0}]},"inferText":"모바일","inferConfidence":1.0},{"boundingPoly":{"vertices":[{"x":512.0,"y":2017.0},{"x":587.0,"y":2017.0},{"x":587.0,"y":2054.0},{"x":512.0,"y":2054.0}]},"inferText":"신분증","inferConfidence":1.0}]}],"boundingPoly":{"vertices":[{"x":42.0,"y":1877.0},{"x":713.0,"y":1877.0},{"x":713.0,"y":2092.0},{"x":42.0,"y":2092.0}]},"inferConfidence":0.9971,"rowSpan":1,"rowIndex":1,"columnSpan":1,"columnIndex":0},{"cellTextLines":[{"boundingPoly":{"vertices":[{"x":168.0,"y":2119.0},{"x":582.0,"y":2119.0},{"x":582.0,"y":2161.0},{"x":168.0,"y":2161.0}]},"inferConfidence":0.9835,"cellWords":[{"boundingPoly":{"vertices":[{"x":168.0,"y":2119.0},{"x":293.0,"y":2119.0},{"x":293.0,"y":2161.0},{"x":168.0,"y":2161.0}]},"inferText":"주민등록증","inferConfidence":1.0},{"boundingPoly":{"vertices":[{"x":289.0,"y":2124.0},{"x":582.0,"y":2124.0},{"x":582.0,"y":2161.0},{"x":289.0,"y":2161.0}]},"inferText":"미발급자(초,중,고학생포함)","inferConfidence":0.967}]},{"boundingPoly":{"vertices":[{"x":112.0,"y":2180.0},{"x":633.0,"y":2180.0},{"x":633.0,"y":2226.0},{"x":112.0,"y":2226.0}]},"inferConfidence":0.99476665,"cellWords":[{"boundingPoly":{"vertices":[{"x":112.0,"y":2180.0},{"x":219.0,"y":2180.0},{"x":219.0,"y":2226.0},{"x":112.0,"y":2226.0}]},"inferText":"청소년증,","inferConfidence":0.9982},{"boundingPoly":{"vertices":[{"x":210.0,"y":2180.0},{"x":442.0,"y":2180.0},{"x":442.0,"y":2226.0},{"x":210.0,"y":2226.0}]},"inferText":"청소년발급신청확인서,","inferConfidence":0.9951},{"boundingPoly":{"vertices":[{"x":438.0,"y":2184.0},{"x":633.0,"y":2184.0},{"x":633.0,"y":2221.0},{"x":438.0,"y":2221.0}]},"inferText":"기간만료 전의 여권","inferConfidence":0.991}]}],"boundingPoly":{"vertices":[{"x":42.0,"y":2092.0},{"x":713.0,"y":2092.0},{"x":713.0,"y":2262.0},{"x":42.0,"y":2262.0}]},"inferConfidence":0.99026,"rowSpan":1,"rowIndex":2,"columnSpan":1,"columnIndex":0},{"cellTextLines":[{"boundingPoly":{"vertices":[{"x":238.0,"y":2287.0},{"x":508.0,"y":2287.0},{"x":508.0,"y":2333.0},{"x":238.0,"y":2333.0}]},"inferConfidence":0.9901,"cellWords":[{"boundingPoly":{"vertices":[{"x":238.0,"y":2287.0},{"x":508.0,"y":2287.0},{"x":508.0,"y":2333.0},{"x":238.0,"y":2333.0}]},"inferText":"외국인 (대한민국 국적 외)","inferConfidence":0.9901}]},{"boundingPoly":{"vertices":[{"x":275.0,"y":2357.0},{"x":480.0,"y":2357.0},{"x":480.0,"y":2394.0},{"x":275.0,"y":2394.0}]},"inferConfidence":0.99685,"cellWords":[{"boundingPoly":{"vertices":[{"x":275.0,"y":2357.0},{"x":428.0,"y":2357.0},{"x":428.0,"y":2394.0},{"x":275.0,"y":2394.0}]},"inferText":"외국인 등록증,","inferConfidence":0.9937},{"boundingPoly":{"vertices":[{"x":419.0,"y":2357.0},{"x":480.0,"y":2357.0},{"x":480.0,"y":2394.0},{"x":419.0,"y":2394.0}]},"inferText":"여권","inferConfidence":1.0}]}],"boundingPoly":{"vertices":[{"x":42.0,"y":2262.0},{"x":713.0,"y":2262.0},{"x":713.0,"y":2431.0},{"x":42.0,"y":2431.0}]},"inferConfidence":0.9946,"rowSpan":1,"rowIndex":3,"columnSpan":1,"columnIndex":0}],"boundingPoly":{"vertices":[{"x":42.0,"y":1819.0},{"x":713.0,"y":1819.0},{"x":713.0,"y":2431.0},{"x":42.0,"y":2431.0}]},"inferConfidence":0.9953025},{"cells":[{"cellTextLines":[{"boundingPoly":{"vertices":[{"x":242.0,"y":2478.0},{"x":508.0,"y":2478.0},{"x":508.0,"y":2524.0},{"x":242.0,"y":2524.0}]},"inferConfidence":0.99965,"cellWords":[{"boundingPoly":{"vertices":[{"x":242.0,"y":2478.0},{"x":391.0,"y":2478.0},{"x":391.0,"y":2524.0},{"x":242.0,"y":2524.0}]},"inferText":"인정되지 않는","inferConfidence":0.9998},{"boundingPoly":{"vertices":[{"x":382.0,"y":2478.0},{"x":508.0,"y":2478.0},{"x":508.0,"y":2524.0},{"x":382.0,"y":2524.0}]},"inferText":"무효신분증","inferConfidence":0.9995}]}],"boundingPoly":{"vertices":[{"x":29.0,"y":2471.0},{"x":707.0,"y":2464.0},{"x":707.0,"y":2523.0},{"x":29.0,"y":2529.0}]},"inferConfidence":0.99965,"rowSpan":1,"rowIndex":0,"columnSpan":1,"columnIndex":0},{"cellTextLines":[{"boundingPoly":{"vertices":[{"x":88.0,"y":2566.0},{"x":661.0,"y":2566.0},{"x":661.0,"y":2608.0},{"x":88.0,"y":2608.0}]},"inferConfidence":0.99908,"cellWords":[{"boundingPoly":{"vertices":[{"x":88.0,"y":2571.0},{"x":168.0,"y":2571.0},{"x":168.0,"y":2608.0},{"x":88.0,"y":2608.0}]},"inferText":"학생증,","inferConfidence":0.9999},{"boundingPoly":{"vertices":[{"x":163.0,"y":2571.0},{"x":247.0,"y":2571.0},{"x":247.0,"y":2608.0},{"x":163.0,"y":2608.0}]},"inferText":"사원증,","inferConfidence":0.9999},{"boundingPoly":{"vertices":[{"x":242.0,"y":2571.0},{"x":405.0,"y":2571.0},{"x":405.0,"y":2608.0},{"x":242.0,"y":2608.0}]},"inferText":"국민건강보험증,","inferConfidence":0.997},{"boundingPoly":{"vertices":[{"x":396.0,"y":2571.0},{"x":545.0,"y":2571.0},{"x":545.0,"y":2608.0},{"x":396.0,"y":2608.0}]},"inferText":"주민등록등본,","inferConfidence":0.9991},{"boundingPoly":{"vertices":[{"x":540.0,"y":2566.0},{"x":661.0,"y":2566.0},{"x":661.0,"y":2608.0},{"x":540.0,"y":2608.0}]},"inferText":"의료보험증,","inferConfidence":0.9995}]},{"boundingPoly":{"vertices":[{"x":79.0,"y":2603.0},{"x":671.0,"y":2603.0},{"x":671.0,"y":2650.0},{"x":79.0,"y":2650.0}]},"inferConfidence":0.9996714,"cellWords":[{"boundingPoly":{"vertices":[{"x":79.0,"y":2603.0},{"x":186.0,"y":2603.0},{"x":186.0,"y":2650.0},{"x":79.0,"y":2650.0}]},"inferText":"공무원증,","inferConfidence":0.9985},{"boundingPoly":{"vertices":[{"x":182.0,"y":2608.0},{"x":256.0,"y":2608.0},{"x":256.0,"y":2645.0},{"x":182.0,"y":2645.0}]},"inferText":"기간이","inferConfidence":1.0},{"boundingPoly":{"vertices":[{"x":251.0,"y":2608.0},{"x":326.0,"y":2608.0},{"x":326.0,"y":2645.0},{"x":251.0,"y":2645.0}]},"inferText":"만료된","inferConfidence":0.9996},{"boundingPoly":{"vertices":[{"x":317.0,"y":2608.0},{"x":382.0,"y":2608.0},{"x":382.0,"y":2645.0},{"x":317.0,"y":2645.0}]},"inferText":"여권,","inferConfidence":1.0},{"boundingPoly":{"vertices":[{"x":373.0,"y":2608.0},{"x":522.0,"y":2608.0},{"x":522.0,"y":2645.0},{"x":373.0,"y":2645.0}]},"inferText":"기간이 만료된","inferConfidence":0.9999},{"boundingPoly":{"vertices":[{"x":512.0,"y":2608.0},{"x":591.0,"y":2608.0},{"x":591.0,"y":2645.0},{"x":512.0,"y":2645.0}]},"inferText":"외국인","inferConfidence":0.9998},{"boundingPoly":{"vertices":[{"x":587.0,"y":2608.0},{"x":671.0,"y":2608.0},{"x":671.0,"y":2650.0},{"x":587.0,"y":2650.0}]},"inferText":"등록증,","inferConfidence":0.9999}]},{"boundingPoly":{"vertices":[{"x":75.0,"y":2650.0},{"x":675.0,"y":2650.0},{"x":675.0,"y":2687.0},{"x":75.0,"y":2687.0}]},"inferConfidence":0.993625,"cellWords":[{"boundingPoly":{"vertices":[{"x":75.0,"y":2650.0},{"x":177.0,"y":2650.0},{"x":177.0,"y":2687.0},{"x":75.0,"y":2687.0}]},"inferText":"기관장의","inferConfidence":0.9947},{"boundingPoly":{"vertices":[{"x":172.0,"y":2650.0},{"x":228.0,"y":2650.0},{"x":228.0,"y":2683.0},{"x":172.0,"y":2683.0}]},"inferText":"관인","inferConfidence":0.9994},{"boundingPoly":{"vertices":[{"x":224.0,"y":2650.0},{"x":251.0,"y":2650.0},{"x":251.0,"y":2683.0},{"x":224.0,"y":2683.0}]},"inferText":"및","inferConfidence":1.0},{"boundingPoly":{"vertices":[{"x":247.0,"y":2650.0},{"x":303.0,"y":2650.0},{"x":303.0,"y":2687.0},{"x":247.0,"y":2687.0}]},"inferText":"투명","inferConfidence":1.0},{"boundingPoly":{"vertices":[{"x":298.0,"y":2650.0},{"x":484.0,"y":2650.0},{"x":484.0,"y":2687.0},{"x":298.0,"y":2687.0}]},"inferText":"스티커가 부착되어","inferConfidence":0.9614},{"boundingPoly":{"vertices":[{"x":475.0,"y":2650.0},{"x":531.0,"y":2650.0},{"x":531.0,"y":2687.0},{"x":475.0,"y":2687.0}]},"inferText":"있지","inferConfidence":0.9944},{"boundingPoly":{"vertices":[{"x":526.0,"y":2650.0},{"x":582.0,"y":2650.0},{"x":582.0,"y":2687.0},{"x":526.0,"y":2687.0}]},"inferText":"않은","inferConfidence":0.9997},{"boundingPoly":{"vertices":[{"x":577.0,"y":2650.0},{"x":675.0,"y":2650.0},{"x":675.0,"y":2687.0},{"x":577.0,"y":2687.0}]},"inferText":"청소년증","inferConfidence":0.9994}]},{"boundingPoly":{"vertices":[{"x":93.0,"y":2683.0},{"x":661.0,"y":2683.0},{"x":661.0,"y":2729.0},{"x":93.0,"y":2729.0}]},"inferConfidence":0.9998889,"cellWords":[{"boundingPoly":{"vertices":[{"x":93.0,"y":2687.0},{"x":149.0,"y":2687.0},{"x":149.0,"y":2729.0},{"x":93.0,"y":2729.0}]},"inferText":"발급","inferConfidence":0.9999},{"boundingPoly":{"vertices":[{"x":140.0,"y":2683.0},{"x":261.0,"y":2683.0},{"x":261.0,"y":2729.0},{"x":140.0,"y":2729.0}]},"inferText":"신청확인서,","inferConfidence":0.9998},{"boundingPoly":{"vertices":[{"x":261.0,"y":2687.0},{"x":317.0,"y":2687.0},{"x":317.0,"y":2729.0},{"x":261.0,"y":2729.0}]},"inferText":"캡처","inferConfidence":1.0},{"boundingPoly":{"vertices":[{"x":307.0,"y":2692.0},{"x":340.0,"y":2692.0},{"x":340.0,"y":2720.0},{"x":307.0,"y":2720.0}]},"inferText":"된","inferConfidence":0.9998},{"boundingPoly":{"vertices":[{"x":335.0,"y":2683.0},{"x":414.0,"y":2683.0},{"x":414.0,"y":2729.0},{"x":335.0,"y":2729.0}]},"inferText":"모바일","inferConfidence":1.0},{"boundingPoly":{"vertices":[{"x":405.0,"y":2683.0},{"x":484.0,"y":2683.0},{"x":484.0,"y":2729.0},{"x":405.0,"y":2729.0}]},"inferText":"신분증","inferConfidence":1.0},{"boundingPoly":{"vertices":[{"x":480.0,"y":2687.0},{"x":508.0,"y":2687.0},{"x":508.0,"y":2720.0},{"x":480.0,"y":2720.0}]},"inferText":"등","inferConfidence":0.9998},{"boundingPoly":{"vertices":[{"x":508.0,"y":2687.0},{"x":587.0,"y":2687.0},{"x":587.0,"y":2729.0},{"x":508.0,"y":2729.0}]},"inferText":"위 유효","inferConfidence":0.9997},{"boundingPoly":{"vertices":[{"x":577.0,"y":2683.0},{"x":661.0,"y":2683.0},{"x":661.0,"y":2729.0},{"x":577.0,"y":2729.0}]},"inferText":"신분증","inferConfidence":1.0}]},{"boundingPoly":{"vertices":[{"x":228.0,"y":2729.0},{"x":526.0,"y":2729.0},{"x":526.0,"y":2766.0},{"x":228.0,"y":2766.0}]},"inferConfidence":0.9996,"cellWords":[{"boundingPoly":{"vertices":[{"x":228.0,"y":2729.0},{"x":307.0,"y":2729.0},{"x":307.0,"y":2766.0},{"x":228.0,"y":2766.0}]},"inferText":"조건에","inferConfidence":1.0},{"boundingPoly":{"vertices":[{"x":298.0,"y":2729.0},{"x":354.0,"y":2729.0},{"x":354.0,"y":2766.0},{"x":298.0,"y":2766.0}]},"inferText":"맞지","inferConfidence":0.998},{"boundingPoly":{"vertices":[{"x":345.0,"y":2729.0},{"x":405.0,"y":2729.0},{"x":405.0,"y":2766.0},{"x":345.0,"y":2766.0}]},"inferText":"않는","inferConfidence":1.0},{"boundingPoly":{"vertices":[{"x":396.0,"y":2729.0},{"x":452.0,"y":2729.0},{"x":452.0,"y":2766.0},{"x":396.0,"y":2766.0}]},"inferText":"모든","inferConfidence":1.0},{"boundingPoly":{"vertices":[{"x":447.0,"y":2729.0},{"x":526.0,"y":2729.0},{"x":526.0,"y":2766.0},{"x":447.0,"y":2766.0}]},"inferText":"신분증","inferConfidence":1.0}]}],"boundingPoly":{"vertices":[{"x":29.0,"y":2529.0},{"x":707.0,"y":2523.0},{"x":707.0,"y":2797.0},{"x":42.0,"y":2797.0}]},"inferConfidence":0.9982088,"rowSpan":1,"rowIndex":1,"columnSpan":1,"columnIndex":0}],"boundingPoly":{"vertices":[{"x":29.0,"y":2471.0},{"x":707.0,"y":2464.0},{"x":707.0,"y":2797.0},{"x":42.0,"y":2797.0}]},"inferConfidence":0.99892944}],"fields":[{"valueType":"ALL","boundingPoly":{"vertices":[{"x":126.0,"y":47.0},{"x":233.0,"y":47.0},{"x":233.0,"y":121.0},{"x":126.0,"y":121.0}]},"inferText":"티켓","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":210.0,"y":47.0},{"x":312.0,"y":47.0},{"x":312.0,"y":116.0},{"x":210.0,"y":116.0}]},"inferText":"배송","inferConfidence":0.9964,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":303.0,"y":51.0},{"x":359.0,"y":51.0},{"x":359.0,"y":112.0},{"x":303.0,"y":112.0}]},"inferText":"및","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":345.0,"y":47.0},{"x":452.0,"y":47.0},{"x":452.0,"y":121.0},{"x":345.0,"y":121.0}]},"inferText":"현장","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":433.0,"y":51.0},{"x":531.0,"y":51.0},{"x":531.0,"y":112.0},{"x":433.0,"y":112.0}]},"inferText":"수령","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":517.0,"y":47.0},{"x":624.0,"y":47.0},{"x":624.0,"y":121.0},{"x":517.0,"y":121.0}]},"inferText":"안내","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":279.0,"y":168.0},{"x":405.0,"y":168.0},{"x":405.0,"y":219.0},{"x":279.0,"y":219.0}]},"inferText":"티켓 배송","inferConfidence":0.9446,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":401.0,"y":172.0},{"x":466.0,"y":172.0},{"x":466.0,"y":214.0},{"x":401.0,"y":214.0}]},"inferText":"안내","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":261.0},{"x":168.0,"y":261.0},{"x":168.0,"y":298.0},{"x":61.0,"y":298.0}]},"inferText":"공연당일","inferConfidence":0.9658,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":158.0,"y":261.0},{"x":238.0,"y":261.0},{"x":238.0,"y":298.0},{"x":158.0,"y":298.0}]},"inferText":"혼잡을","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":228.0,"y":261.0},{"x":289.0,"y":261.0},{"x":289.0,"y":298.0},{"x":228.0,"y":298.0}]},"inferText":"막기","inferConfidence":0.9971,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":279.0,"y":261.0},{"x":331.0,"y":261.0},{"x":331.0,"y":293.0},{"x":279.0,"y":293.0}]},"inferText":"위해","inferConfidence":0.9971,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":326.0,"y":261.0},{"x":475.0,"y":261.0},{"x":475.0,"y":298.0},{"x":326.0,"y":298.0}]},"inferText":"예매 초기에는","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":470.0,"y":261.0},{"x":526.0,"y":261.0},{"x":526.0,"y":298.0},{"x":470.0,"y":298.0}]},"inferText":"일괄","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":517.0,"y":261.0},{"x":573.0,"y":261.0},{"x":573.0,"y":298.0},{"x":517.0,"y":298.0}]},"inferText":"배송","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":568.0,"y":261.0},{"x":671.0,"y":261.0},{"x":671.0,"y":298.0},{"x":568.0,"y":298.0}]},"inferText":"방식으로","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":298.0},{"x":186.0,"y":298.0},{"x":186.0,"y":340.0},{"x":61.0,"y":340.0}]},"inferText":"운영합니다.","inferConfidence":0.9995,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":177.0,"y":298.0},{"x":242.0,"y":298.0},{"x":242.0,"y":335.0},{"x":177.0,"y":335.0}]},"inferText":"(일괄","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":233.0,"y":298.0},{"x":317.0,"y":298.0},{"x":317.0,"y":335.0},{"x":233.0,"y":335.0}]},"inferText":"배송일:","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":307.0,"y":298.0},{"x":354.0,"y":298.0},{"x":354.0,"y":331.0},{"x":307.0,"y":331.0}]},"inferText":"6월","inferConfidence":0.9953,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":345.0,"y":293.0},{"x":531.0,"y":293.0},{"x":531.0,"y":340.0},{"x":345.0,"y":340.0}]},"inferText":"9일(월)부터 순차","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":522.0,"y":298.0},{"x":587.0,"y":298.0},{"x":587.0,"y":335.0},{"x":522.0,"y":335.0}]},"inferText":"배송)","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":368.0},{"x":116.0,"y":368.0},{"x":116.0,"y":405.0},{"x":61.0,"y":405.0}]},"inferText":"일괄","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":112.0,"y":368.0},{"x":168.0,"y":368.0},{"x":168.0,"y":405.0},{"x":112.0,"y":405.0}]},"inferText":"배송","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":158.0,"y":368.0},{"x":279.0,"y":368.0},{"x":279.0,"y":405.0},{"x":158.0,"y":405.0}]},"inferText":"티켓은 5월","inferConfidence":0.9973,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":270.0,"y":368.0},{"x":368.0,"y":368.0},{"x":368.0,"y":405.0},{"x":270.0,"y":405.0}]},"inferText":"28일(수)","inferConfidence":0.9931,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":368.0,"y":368.0},{"x":536.0,"y":368.0},{"x":536.0,"y":405.0},{"x":368.0,"y":405.0}]},"inferText":"10:00(KST)까지","inferConfidence":0.9989,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":531.0,"y":368.0},{"x":657.0,"y":368.0},{"x":657.0,"y":405.0},{"x":531.0,"y":405.0}]},"inferText":"예매 완료된","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":405.0},{"x":140.0,"y":405.0},{"x":140.0,"y":442.0},{"x":61.0,"y":442.0}]},"inferText":"건부터","inferConfidence":0.9989,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":128.0,"y":409.0},{"x":256.0,"y":402.0},{"x":258.0,"y":442.0},{"x":131.0,"y":450.0}]},"inferText":"순차적으로","inferConfidence":0.9994,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":247.0,"y":405.0},{"x":326.0,"y":405.0},{"x":326.0,"y":447.0},{"x":247.0,"y":447.0}]},"inferText":"배송될","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":317.0,"y":405.0},{"x":419.0,"y":405.0},{"x":419.0,"y":447.0},{"x":317.0,"y":447.0}]},"inferText":"예정이며,","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":414.0,"y":405.0},{"x":470.0,"y":405.0},{"x":470.0,"y":442.0},{"x":414.0,"y":442.0}]},"inferText":"일괄","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":466.0,"y":405.0},{"x":540.0,"y":405.0},{"x":540.0,"y":442.0},{"x":466.0,"y":442.0}]},"inferText":"배송일","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":531.0,"y":405.0},{"x":591.0,"y":405.0},{"x":591.0,"y":442.0},{"x":531.0,"y":442.0}]},"inferText":"이후","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":447.0},{"x":182.0,"y":447.0},{"x":182.0,"y":484.0},{"x":61.0,"y":484.0}]},"inferText":"예매하시는","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":177.0,"y":447.0},{"x":251.0,"y":447.0},{"x":251.0,"y":484.0},{"x":177.0,"y":484.0}]},"inferText":"분들은","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":247.0,"y":447.0},{"x":303.0,"y":447.0},{"x":303.0,"y":484.0},{"x":247.0,"y":484.0}]},"inferText":"현장","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":298.0,"y":447.0},{"x":373.0,"y":447.0},{"x":373.0,"y":484.0},{"x":298.0,"y":484.0}]},"inferText":"수령만","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":363.0,"y":447.0},{"x":494.0,"y":447.0},{"x":494.0,"y":484.0},{"x":363.0,"y":484.0}]},"inferText":"가능합니다.","inferConfidence":0.9992,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":508.0},{"x":140.0,"y":508.0},{"x":140.0,"y":550.0},{"x":61.0,"y":550.0}]},"inferText":"배송지","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":130.0,"y":508.0},{"x":210.0,"y":508.0},{"x":210.0,"y":545.0},{"x":130.0,"y":545.0}]},"inferText":"변경은","inferConfidence":0.9785,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":200.0,"y":512.0},{"x":247.0,"y":512.0},{"x":247.0,"y":540.0},{"x":200.0,"y":540.0}]},"inferText":"6월","inferConfidence":0.9994,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":242.0,"y":508.0},{"x":359.0,"y":508.0},{"x":359.0,"y":550.0},{"x":242.0,"y":550.0}]},"inferText":"9일(월) 낮","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":354.0,"y":507.0},{"x":523.0,"y":512.0},{"x":521.0,"y":550.0},{"x":353.0,"y":545.0}]},"inferText":"12:00(KST)까지","inferConfidence":0.9984,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":517.0,"y":508.0},{"x":643.0,"y":508.0},{"x":643.0,"y":550.0},{"x":517.0,"y":550.0}]},"inferText":"가능합니다.","inferConfidence":0.9986,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":550.0},{"x":121.0,"y":550.0},{"x":121.0,"y":587.0},{"x":61.0,"y":587.0}]},"inferText":"티켓","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":112.0,"y":550.0},{"x":196.0,"y":550.0},{"x":196.0,"y":587.0},{"x":112.0,"y":587.0}]},"inferText":"구매 전","inferConfidence":0.9997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":191.0,"y":550.0},{"x":242.0,"y":550.0},{"x":242.0,"y":587.0},{"x":191.0,"y":587.0}]},"inferText":"수령","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":238.0,"y":550.0},{"x":345.0,"y":550.0},{"x":345.0,"y":587.0},{"x":238.0,"y":587.0}]},"inferText":"정보(배송","inferConfidence":0.9997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":340.0,"y":550.0},{"x":396.0,"y":550.0},{"x":396.0,"y":587.0},{"x":340.0,"y":587.0}]},"inferText":"주소,","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":396.0,"y":550.0},{"x":470.0,"y":550.0},{"x":470.0,"y":587.0},{"x":396.0,"y":587.0}]},"inferText":"수령인,","inferConfidence":0.9997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":466.0,"y":550.0},{"x":573.0,"y":550.0},{"x":573.0,"y":587.0},{"x":466.0,"y":587.0}]},"inferText":"연락처)가","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":568.0,"y":550.0},{"x":666.0,"y":550.0},{"x":666.0,"y":587.0},{"x":568.0,"y":587.0}]},"inferText":"정확한지","inferConfidence":0.9999,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":591.0},{"x":116.0,"y":591.0},{"x":116.0,"y":619.0},{"x":61.0,"y":619.0}]},"inferText":"다시","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":112.0,"y":587.0},{"x":168.0,"y":587.0},{"x":168.0,"y":624.0},{"x":112.0,"y":624.0}]},"inferText":"한번","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":158.0,"y":587.0},{"x":261.0,"y":587.0},{"x":261.0,"y":624.0},{"x":158.0,"y":624.0}]},"inferText":"확인하여","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":256.0,"y":591.0},{"x":382.0,"y":591.0},{"x":382.0,"y":629.0},{"x":256.0,"y":629.0}]},"inferText":"주시기 바랍","inferConfidence":0.9995,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":377.0,"y":587.0},{"x":475.0,"y":587.0},{"x":475.0,"y":624.0},{"x":377.0,"y":624.0}]},"inferText":".다. 배송","inferConfidence":0.871,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":470.0,"y":591.0},{"x":550.0,"y":591.0},{"x":550.0,"y":624.0},{"x":470.0,"y":624.0}]},"inferText":"후에는","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":540.0,"y":587.0},{"x":596.0,"y":587.0},{"x":596.0,"y":624.0},{"x":540.0,"y":624.0}]},"inferText":"수령","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":591.0,"y":587.0},{"x":666.0,"y":587.0},{"x":666.0,"y":624.0},{"x":591.0,"y":624.0}]},"inferText":"정보의","inferConfidence":0.9854,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":629.0},{"x":140.0,"y":629.0},{"x":140.0,"y":666.0},{"x":61.0,"y":666.0}]},"inferText":"변경이","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":130.0,"y":629.0},{"x":233.0,"y":629.0},{"x":233.0,"y":666.0},{"x":130.0,"y":666.0}]},"inferText":"어려우며,","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":228.0,"y":629.0},{"x":284.0,"y":629.0},{"x":284.0,"y":666.0},{"x":228.0,"y":666.0}]},"inferText":"반송","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":284.0,"y":633.0},{"x":303.0,"y":633.0},{"x":303.0,"y":657.0},{"x":284.0,"y":657.0}]},"inferText":"후","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":307.0,"y":629.0},{"x":512.0,"y":629.0},{"x":512.0,"y":666.0},{"x":307.0,"y":666.0}]},"inferText":"재배송은불가합니다.","inferConfidence":0.9995,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":508.0,"y":629.0},{"x":577.0,"y":629.0},{"x":577.0,"y":666.0},{"x":508.0,"y":666.0}]},"inferText":"(현장","inferConfidence":0.9996,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":568.0,"y":629.0},{"x":703.0,"y":629.0},{"x":703.0,"y":666.0},{"x":568.0,"y":666.0}]},"inferText":"수령만 가능)","inferConfidence":0.9997,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":694.0},{"x":121.0,"y":694.0},{"x":121.0,"y":736.0},{"x":61.0,"y":736.0}]},"inferText":"티켓","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":112.0,"y":694.0},{"x":168.0,"y":694.0},{"x":168.0,"y":736.0},{"x":112.0,"y":736.0}]},"inferText":"수령","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":163.0,"y":699.0},{"x":196.0,"y":699.0},{"x":196.0,"y":731.0},{"x":163.0,"y":731.0}]},"inferText":"후,","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":191.0,"y":694.0},{"x":317.0,"y":694.0},{"x":317.0,"y":736.0},{"x":191.0,"y":736.0}]},"inferText":"온라인상에","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":307.0,"y":694.0},{"x":382.0,"y":694.0},{"x":382.0,"y":736.0},{"x":307.0,"y":736.0}]},"inferText":"티켓의","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":377.0,"y":699.0},{"x":475.0,"y":699.0},{"x":475.0,"y":736.0},{"x":377.0,"y":736.0}]},"inferText":"이미지를","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":470.0,"y":699.0},{"x":526.0,"y":699.0},{"x":526.0,"y":736.0},{"x":470.0,"y":736.0}]},"inferText":"절대","inferConfidence":0.9981,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":517.0,"y":694.0},{"x":591.0,"y":694.0},{"x":591.0,"y":731.0},{"x":517.0,"y":731.0}]},"inferText":"올리지","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":582.0,"y":689.0},{"x":685.0,"y":689.0},{"x":685.0,"y":736.0},{"x":582.0,"y":736.0}]},"inferText":"않으시길","inferConfidence":0.9996,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":736.0},{"x":144.0,"y":736.0},{"x":144.0,"y":773.0},{"x":61.0,"y":773.0}]},"inferText":"바라며,","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":140.0,"y":736.0},{"x":196.0,"y":736.0},{"x":196.0,"y":773.0},{"x":140.0,"y":773.0}]},"inferText":"불법","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":191.0,"y":736.0},{"x":265.0,"y":736.0},{"x":265.0,"y":773.0},{"x":191.0,"y":773.0}]},"inferText":"복제를","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":256.0,"y":731.0},{"x":317.0,"y":731.0},{"x":317.0,"y":778.0},{"x":256.0,"y":778.0}]},"inferText":"통한","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":307.0,"y":736.0},{"x":363.0,"y":736.0},{"x":363.0,"y":773.0},{"x":307.0,"y":773.0}]},"inferText":"가짜","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":354.0,"y":736.0},{"x":456.0,"y":736.0},{"x":456.0,"y":773.0},{"x":354.0,"y":773.0}]},"inferText":"티켓으로","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":447.0,"y":736.0},{"x":526.0,"y":736.0},{"x":526.0,"y":773.0},{"x":447.0,"y":773.0}]},"inferText":"위조될","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":517.0,"y":736.0},{"x":596.0,"y":736.0},{"x":596.0,"y":773.0},{"x":517.0,"y":773.0}]},"inferText":"소지가","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":587.0,"y":736.0},{"x":666.0,"y":736.0},{"x":666.0,"y":773.0},{"x":587.0,"y":773.0}]},"inferText":"있으니","inferConfidence":0.9997,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":65.0,"y":782.0},{"x":88.0,"y":782.0},{"x":88.0,"y":801.0},{"x":65.0,"y":801.0}]},"inferText":"이","inferConfidence":0.7148,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":782.0},{"x":121.0,"y":782.0},{"x":121.0,"y":806.0},{"x":98.0,"y":806.0}]},"inferText":"점","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":112.0,"y":767.0},{"x":308.0,"y":773.0},{"x":307.0,"y":816.0},{"x":111.0,"y":810.0}]},"inferText":"유의하시어 피해를","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":303.0,"y":778.0},{"x":354.0,"y":778.0},{"x":354.0,"y":815.0},{"x":303.0,"y":815.0}]},"inferText":"입지","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":349.0,"y":778.0},{"x":447.0,"y":778.0},{"x":447.0,"y":815.0},{"x":349.0,"y":815.0}]},"inferText":"않으시길","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":438.0,"y":778.0},{"x":545.0,"y":778.0},{"x":545.0,"y":815.0},{"x":438.0,"y":815.0}]},"inferText":"바랍니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":838.0},{"x":116.0,"y":838.0},{"x":116.0,"y":876.0},{"x":61.0,"y":876.0}]},"inferText":"위조","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":112.0,"y":838.0},{"x":186.0,"y":838.0},{"x":186.0,"y":876.0},{"x":112.0,"y":876.0}]},"inferText":"티켓을","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":177.0,"y":834.0},{"x":279.0,"y":834.0},{"x":279.0,"y":876.0},{"x":177.0,"y":876.0}]},"inferText":"판매하는","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":279.0,"y":843.0},{"x":303.0,"y":843.0},{"x":303.0,"y":871.0},{"x":279.0,"y":871.0}]},"inferText":"등","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":303.0,"y":838.0},{"x":419.0,"y":838.0},{"x":419.0,"y":876.0},{"x":303.0,"y":876.0}]},"inferText":"위법행위로","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":414.0,"y":838.0},{"x":512.0,"y":838.0},{"x":512.0,"y":876.0},{"x":414.0,"y":876.0}]},"inferText":"추정되는","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":508.0,"y":838.0},{"x":559.0,"y":838.0},{"x":559.0,"y":876.0},{"x":508.0,"y":876.0}]},"inferText":"경우","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":554.0,"y":838.0},{"x":615.0,"y":838.0},{"x":615.0,"y":876.0},{"x":554.0,"y":876.0}]},"inferText":"주최,","inferConfidence":0.997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":610.0,"y":838.0},{"x":666.0,"y":838.0},{"x":666.0,"y":876.0},{"x":610.0,"y":876.0}]},"inferText":"주관","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":657.0,"y":838.0},{"x":713.0,"y":838.0},{"x":713.0,"y":876.0},{"x":657.0,"y":876.0}]},"inferText":"측의","inferConfidence":0.9995,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":876.0},{"x":163.0,"y":876.0},{"x":163.0,"y":917.0},{"x":61.0,"y":917.0}]},"inferText":"권한으로","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":154.0,"y":876.0},{"x":251.0,"y":876.0},{"x":251.0,"y":917.0},{"x":154.0,"y":917.0}]},"inferText":"사전통보","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":247.0,"y":876.0},{"x":303.0,"y":876.0},{"x":303.0,"y":917.0},{"x":247.0,"y":917.0}]},"inferText":"없이","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":298.0,"y":880.0},{"x":349.0,"y":880.0},{"x":349.0,"y":913.0},{"x":298.0,"y":913.0}]},"inferText":"거래","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":345.0,"y":876.0},{"x":438.0,"y":876.0},{"x":438.0,"y":917.0},{"x":345.0,"y":917.0}]},"inferText":"당사자의","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":433.0,"y":876.0},{"x":512.0,"y":876.0},{"x":512.0,"y":913.0},{"x":433.0,"y":913.0}]},"inferText":"티켓은","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":503.0,"y":876.0},{"x":657.0,"y":876.0},{"x":657.0,"y":917.0},{"x":503.0,"y":917.0}]},"inferText":"무효처리 되며,","inferConfidence":0.9962,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":917.0},{"x":168.0,"y":917.0},{"x":168.0,"y":955.0},{"x":61.0,"y":955.0}]},"inferText":"공연당일","inferConfidence":0.8953,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":163.0,"y":917.0},{"x":238.0,"y":917.0},{"x":238.0,"y":955.0},{"x":163.0,"y":955.0}]},"inferText":"현장에","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":233.0,"y":917.0},{"x":307.0,"y":917.0},{"x":307.0,"y":955.0},{"x":233.0,"y":955.0}]},"inferText":"티켓을","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":303.0,"y":917.0},{"x":484.0,"y":917.0},{"x":484.0,"y":955.0},{"x":303.0,"y":955.0}]},"inferText":"소지하였을지라도","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":480.0,"y":917.0},{"x":536.0,"y":917.0},{"x":536.0,"y":955.0},{"x":480.0,"y":955.0}]},"inferText":"입장","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":526.0,"y":917.0},{"x":577.0,"y":917.0},{"x":577.0,"y":950.0},{"x":526.0,"y":950.0}]},"inferText":"불가","inferConfidence":0.9989,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":568.0,"y":913.0},{"x":671.0,"y":913.0},{"x":671.0,"y":955.0},{"x":568.0,"y":955.0}]},"inferText":"처리됨을","inferConfidence":0.9986,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":959.0},{"x":140.0,"y":959.0},{"x":140.0,"y":997.0},{"x":61.0,"y":997.0}]},"inferText":"반드시","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":130.0,"y":959.0},{"x":210.0,"y":959.0},{"x":210.0,"y":997.0},{"x":130.0,"y":997.0}]},"inferText":"상기해","inferConfidence":0.9934,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":200.0,"y":959.0},{"x":279.0,"y":959.0},{"x":279.0,"y":997.0},{"x":200.0,"y":997.0}]},"inferText":"주시기","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":270.0,"y":959.0},{"x":377.0,"y":959.0},{"x":377.0,"y":997.0},{"x":270.0,"y":997.0}]},"inferText":"바랍니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":65.0,"y":1020.0},{"x":140.0,"y":1020.0},{"x":140.0,"y":1057.0},{"x":65.0,"y":1057.0}]},"inferText":"고객의","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":130.0,"y":1020.0},{"x":210.0,"y":1020.0},{"x":210.0,"y":1057.0},{"x":130.0,"y":1057.0}]},"inferText":"부재로","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":200.0,"y":1020.0},{"x":256.0,"y":1020.0},{"x":256.0,"y":1057.0},{"x":200.0,"y":1057.0}]},"inferText":"인해","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":251.0,"y":1020.0},{"x":331.0,"y":1020.0},{"x":331.0,"y":1057.0},{"x":251.0,"y":1057.0}]},"inferText":"반송된","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":321.0,"y":1020.0},{"x":447.0,"y":1020.0},{"x":447.0,"y":1057.0},{"x":321.0,"y":1057.0}]},"inferText":"티켓은 현장","inferConfidence":0.9981,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":442.0,"y":1020.0},{"x":564.0,"y":1020.0},{"x":564.0,"y":1057.0},{"x":442.0,"y":1057.0}]},"inferText":"매표소에서","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":554.0,"y":1020.0},{"x":680.0,"y":1020.0},{"x":680.0,"y":1057.0},{"x":554.0,"y":1057.0}]},"inferText":"수령하여야","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":1062.0},{"x":196.0,"y":1062.0},{"x":196.0,"y":1099.0},{"x":61.0,"y":1099.0}]},"inferText":"합니다. 현장","inferConfidence":0.9718,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":186.0,"y":1062.0},{"x":242.0,"y":1062.0},{"x":242.0,"y":1099.0},{"x":186.0,"y":1099.0}]},"inferText":"수령","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":238.0,"y":1066.0},{"x":261.0,"y":1066.0},{"x":261.0,"y":1094.0},{"x":238.0,"y":1094.0}]},"inferText":"시","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":261.0,"y":1062.0},{"x":382.0,"y":1062.0},{"x":382.0,"y":1099.0},{"x":261.0,"y":1099.0}]},"inferText":"증빙서류를","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":373.0,"y":1062.0},{"x":545.0,"y":1062.0},{"x":545.0,"y":1099.0},{"x":373.0,"y":1099.0}]},"inferText":"지참하셔야 티켓","inferConfidence":0.9983,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":536.0,"y":1062.0},{"x":615.0,"y":1062.0},{"x":615.0,"y":1099.0},{"x":536.0,"y":1099.0}]},"inferText":"수령이","inferConfidence":0.9999,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":1099.0},{"x":251.0,"y":1099.0},{"x":251.0,"y":1141.0},{"x":61.0,"y":1141.0}]},"inferText":"가능하오니 아래의","inferConfidence":0.9996,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":247.0,"y":1099.0},{"x":303.0,"y":1099.0},{"x":303.0,"y":1141.0},{"x":247.0,"y":1141.0}]},"inferText":"현장","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":298.0,"y":1099.0},{"x":373.0,"y":1099.0},{"x":373.0,"y":1136.0},{"x":298.0,"y":1136.0}]},"inferText":"수령에","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":368.0,"y":1099.0},{"x":424.0,"y":1099.0},{"x":424.0,"y":1136.0},{"x":368.0,"y":1136.0}]},"inferText":"대한","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":414.0,"y":1099.0},{"x":536.0,"y":1099.0},{"x":536.0,"y":1141.0},{"x":414.0,"y":1141.0}]},"inferText":"안내사항을","inferConfidence":0.9997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":531.0,"y":1099.0},{"x":587.0,"y":1099.0},{"x":587.0,"y":1136.0},{"x":531.0,"y":1136.0}]},"inferText":"확인","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":577.0,"y":1099.0},{"x":722.0,"y":1099.0},{"x":722.0,"y":1141.0},{"x":577.0,"y":1141.0}]},"inferText":"부탁드립니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":65.0,"y":1164.0},{"x":140.0,"y":1164.0},{"x":140.0,"y":1202.0},{"x":65.0,"y":1202.0}]},"inferText":"공연일","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":130.0,"y":1164.0},{"x":210.0,"y":1164.0},{"x":210.0,"y":1202.0},{"x":130.0,"y":1202.0}]},"inferText":"일주일","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":205.0,"y":1164.0},{"x":298.0,"y":1164.0},{"x":298.0,"y":1202.0},{"x":205.0,"y":1202.0}]},"inferText":"전까지도","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":289.0,"y":1164.0},{"x":373.0,"y":1164.0},{"x":373.0,"y":1202.0},{"x":289.0,"y":1202.0}]},"inferText":"배송된","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":363.0,"y":1164.0},{"x":438.0,"y":1164.0},{"x":438.0,"y":1202.0},{"x":363.0,"y":1202.0}]},"inferText":"티켓을","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":433.0,"y":1164.0},{"x":531.0,"y":1164.0},{"x":531.0,"y":1202.0},{"x":433.0,"y":1202.0}]},"inferText":"수령하지","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":522.0,"y":1164.0},{"x":629.0,"y":1164.0},{"x":629.0,"y":1202.0},{"x":522.0,"y":1202.0}]},"inferText":"못했을 시","inferConfidence":0.999,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":1206.0},{"x":121.0,"y":1206.0},{"x":121.0,"y":1239.0},{"x":61.0,"y":1239.0}]},"inferText":"NOL","inferConfidence":0.9806,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":116.0,"y":1202.0},{"x":172.0,"y":1202.0},{"x":172.0,"y":1243.0},{"x":116.0,"y":1243.0}]},"inferText":"티켓","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":168.0,"y":1206.0},{"x":326.0,"y":1206.0},{"x":326.0,"y":1243.0},{"x":168.0,"y":1243.0}]},"inferText":"고객센터(1544-","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":327.0,"y":1198.0},{"x":421.0,"y":1206.0},{"x":418.0,"y":1246.0},{"x":324.0,"y":1239.0}]},"inferText":"1555)로","inferConfidence":0.9989,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":410.0,"y":1202.0},{"x":466.0,"y":1202.0},{"x":466.0,"y":1243.0},{"x":410.0,"y":1243.0}]},"inferText":"문의","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":456.0,"y":1206.0},{"x":564.0,"y":1206.0},{"x":564.0,"y":1243.0},{"x":456.0,"y":1243.0}]},"inferText":"바랍니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":1271.0},{"x":251.0,"y":1271.0},{"x":251.0,"y":1309.0},{"x":61.0,"y":1309.0}]},"inferText":"글로벌 페이지에서","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":242.0,"y":1271.0},{"x":321.0,"y":1271.0},{"x":321.0,"y":1304.0},{"x":242.0,"y":1304.0}]},"inferText":"예매한","inferConfidence":0.999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":317.0,"y":1271.0},{"x":391.0,"y":1271.0},{"x":391.0,"y":1309.0},{"x":317.0,"y":1309.0}]},"inferText":"티켓은","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":382.0,"y":1271.0},{"x":438.0,"y":1271.0},{"x":438.0,"y":1309.0},{"x":382.0,"y":1309.0}]},"inferText":"현장","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":433.0,"y":1271.0},{"x":512.0,"y":1271.0},{"x":512.0,"y":1309.0},{"x":433.0,"y":1309.0}]},"inferText":"수령만","inferConfidence":0.9997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":503.0,"y":1271.0},{"x":629.0,"y":1271.0},{"x":629.0,"y":1309.0},{"x":503.0,"y":1309.0}]},"inferText":"가능합니다.","inferConfidence":0.9989,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":256.0,"y":1369.0},{"x":321.0,"y":1369.0},{"x":321.0,"y":1411.0},{"x":256.0,"y":1411.0}]},"inferText":"현장","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":312.0,"y":1369.0},{"x":498.0,"y":1369.0},{"x":498.0,"y":1411.0},{"x":312.0,"y":1411.0}]},"inferText":"티켓 수령 안내","inferConfidence":0.9818,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":1453.0},{"x":140.0,"y":1453.0},{"x":140.0,"y":1490.0},{"x":61.0,"y":1490.0}]},"inferText":"매표소","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":130.0,"y":1453.0},{"x":186.0,"y":1453.0},{"x":186.0,"y":1490.0},{"x":130.0,"y":1490.0}]},"inferText":"운영","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":182.0,"y":1453.0},{"x":293.0,"y":1453.0},{"x":293.0,"y":1495.0},{"x":182.0,"y":1495.0}]},"inferText":"시간: 공연","inferConfidence":0.9927,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":285.0,"y":1451.0},{"x":453.0,"y":1458.0},{"x":451.0,"y":1494.0},{"x":283.0,"y":1487.0}]},"inferText":"시작 5시간 전 ~","inferConfidence":0.9953,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":447.0,"y":1453.0},{"x":503.0,"y":1453.0},{"x":503.0,"y":1495.0},{"x":447.0,"y":1495.0}]},"inferText":"공연","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":494.0,"y":1453.0},{"x":624.0,"y":1453.0},{"x":624.0,"y":1490.0},{"x":494.0,"y":1490.0}]},"inferText":"시작 후 종료","inferConfidence":0.9998,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":1495.0},{"x":126.0,"y":1495.0},{"x":126.0,"y":1532.0},{"x":61.0,"y":1532.0}]},"inferText":"(현장","inferConfidence":0.9995,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":116.0,"y":1495.0},{"x":196.0,"y":1495.0},{"x":196.0,"y":1532.0},{"x":116.0,"y":1532.0}]},"inferText":"상황에","inferConfidence":0.9913,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":191.0,"y":1495.0},{"x":247.0,"y":1495.0},{"x":247.0,"y":1532.0},{"x":191.0,"y":1532.0}]},"inferText":"따라","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":242.0,"y":1495.0},{"x":317.0,"y":1495.0},{"x":317.0,"y":1532.0},{"x":242.0,"y":1532.0}]},"inferText":"변동될","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":312.0,"y":1495.0},{"x":340.0,"y":1495.0},{"x":340.0,"y":1528.0},{"x":312.0,"y":1528.0}]},"inferText":"수","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":335.0,"y":1495.0},{"x":452.0,"y":1495.0},{"x":452.0,"y":1532.0},{"x":335.0,"y":1532.0}]},"inferText":"있습니다.)","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":1560.0},{"x":116.0,"y":1560.0},{"x":116.0,"y":1597.0},{"x":61.0,"y":1597.0}]},"inferText":"티켓","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":112.0,"y":1560.0},{"x":186.0,"y":1560.0},{"x":186.0,"y":1597.0},{"x":112.0,"y":1597.0}]},"inferText":"수령은","inferConfidence":0.9991,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":186.0,"y":1560.0},{"x":265.0,"y":1560.0},{"x":265.0,"y":1597.0},{"x":186.0,"y":1597.0}]},"inferText":"'예매자","inferConfidence":0.9929,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":261.0,"y":1560.0},{"x":335.0,"y":1560.0},{"x":335.0,"y":1597.0},{"x":261.0,"y":1597.0}]},"inferText":"본인의","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":331.0,"y":1560.0},{"x":382.0,"y":1560.0},{"x":382.0,"y":1597.0},{"x":331.0,"y":1597.0}]},"inferText":"실물","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":377.0,"y":1560.0},{"x":466.0,"y":1560.0},{"x":466.0,"y":1602.0},{"x":377.0,"y":1602.0}]},"inferText":"신분증',","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":461.0,"y":1560.0},{"x":522.0,"y":1560.0},{"x":522.0,"y":1597.0},{"x":461.0,"y":1597.0}]},"inferText":"'예매","inferConfidence":0.9956,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":512.0,"y":1560.0},{"x":601.0,"y":1560.0},{"x":601.0,"y":1597.0},{"x":512.0,"y":1597.0}]},"inferText":"내역서',","inferConfidence":0.9977,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":1602.0},{"x":140.0,"y":1602.0},{"x":140.0,"y":1639.0},{"x":61.0,"y":1639.0}]},"inferText":"그리고","inferConfidence":0.999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":135.0,"y":1602.0},{"x":242.0,"y":1602.0},{"x":242.0,"y":1639.0},{"x":135.0,"y":1639.0}]},"inferText":"'추가 증빙","inferConfidence":0.9891,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":238.0,"y":1602.0},{"x":317.0,"y":1602.0},{"x":317.0,"y":1639.0},{"x":238.0,"y":1639.0}]},"inferText":"서류'를","inferConfidence":0.9979,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":312.0,"y":1602.0},{"x":387.0,"y":1602.0},{"x":387.0,"y":1639.0},{"x":312.0,"y":1639.0}]},"inferText":"확인한","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":382.0,"y":1602.0},{"x":554.0,"y":1602.0},{"x":554.0,"y":1639.0},{"x":382.0,"y":1639.0}]},"inferText":"후 이루어집니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":1663.0},{"x":251.0,"y":1663.0},{"x":251.0,"y":1705.0},{"x":61.0,"y":1705.0}]},"inferText":"예매자 본인이라도","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":242.0,"y":1663.0},{"x":298.0,"y":1663.0},{"x":298.0,"y":1705.0},{"x":242.0,"y":1705.0}]},"inferText":"본인","inferConfidence":0.9874,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":293.0,"y":1663.0},{"x":368.0,"y":1663.0},{"x":368.0,"y":1705.0},{"x":293.0,"y":1705.0}]},"inferText":"명의와","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":363.0,"y":1663.0},{"x":461.0,"y":1663.0},{"x":461.0,"y":1705.0},{"x":363.0,"y":1705.0}]},"inferText":"일치하는","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":456.0,"y":1663.0},{"x":554.0,"y":1663.0},{"x":554.0,"y":1705.0},{"x":456.0,"y":1705.0}]},"inferText":"신분증을","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":545.0,"y":1663.0},{"x":624.0,"y":1663.0},{"x":624.0,"y":1705.0},{"x":545.0,"y":1705.0}]},"inferText":"확인할","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":619.0,"y":1663.0},{"x":703.0,"y":1663.0},{"x":703.0,"y":1705.0},{"x":619.0,"y":1705.0}]},"inferText":"수 없는","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":1705.0},{"x":121.0,"y":1705.0},{"x":121.0,"y":1742.0},{"x":61.0,"y":1742.0}]},"inferText":"경우,","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":107.0,"y":1700.0},{"x":242.0,"y":1700.0},{"x":242.0,"y":1746.0},{"x":107.0,"y":1746.0}]},"inferText":", 티켓 수령이","inferConfidence":0.9979,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":233.0,"y":1705.0},{"x":354.0,"y":1705.0},{"x":354.0,"y":1742.0},{"x":233.0,"y":1742.0}]},"inferText":"불가하오니","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":345.0,"y":1705.0},{"x":540.0,"y":1705.0},{"x":540.0,"y":1742.0},{"x":345.0,"y":1742.0}]},"inferText":"반드시 본인 확인이","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":536.0,"y":1705.0},{"x":615.0,"y":1705.0},{"x":615.0,"y":1742.0},{"x":536.0,"y":1742.0}]},"inferText":"가능한","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":605.0,"y":1705.0},{"x":661.0,"y":1705.0},{"x":661.0,"y":1742.0},{"x":605.0,"y":1742.0}]},"inferText":"실물","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":1746.0},{"x":163.0,"y":1746.0},{"x":163.0,"y":1784.0},{"x":61.0,"y":1784.0}]},"inferText":"신분증을","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":154.0,"y":1746.0},{"x":373.0,"y":1746.0},{"x":373.0,"y":1784.0},{"x":154.0,"y":1784.0}]},"inferText":"지참하시기 바랍니다.","inferConfidence":0.9999,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":228.0,"y":1830.0},{"x":405.0,"y":1830.0},{"x":405.0,"y":1868.0},{"x":228.0,"y":1868.0}]},"inferText":"본인 확인 가능한","inferConfidence":0.9988,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":396.0,"y":1830.0},{"x":526.0,"y":1830.0},{"x":526.0,"y":1868.0},{"x":396.0,"y":1868.0}]},"inferText":"유효 신분증","inferConfidence":0.9997,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":270.0,"y":1914.0},{"x":391.0,"y":1914.0},{"x":391.0,"y":1951.0},{"x":270.0,"y":1951.0}]},"inferText":"주민등록증","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":382.0,"y":1914.0},{"x":484.0,"y":1914.0},{"x":484.0,"y":1951.0},{"x":382.0,"y":1951.0}]},"inferText":"기발급자","inferConfidence":0.9997,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":154.0,"y":1970.0},{"x":284.0,"y":1970.0},{"x":284.0,"y":2017.0},{"x":154.0,"y":2017.0}]},"inferText":"주민등록증,","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":270.0,"y":1970.0},{"x":396.0,"y":1970.0},{"x":396.0,"y":2017.0},{"x":270.0,"y":2017.0}]},"inferText":", 운전면허증,","inferConfidence":0.9739,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":391.0,"y":1975.0},{"x":494.0,"y":1975.0},{"x":494.0,"y":2012.0},{"x":391.0,"y":2012.0}]},"inferText":"기간만료","inferConfidence":0.9974,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":484.0,"y":1975.0},{"x":540.0,"y":1975.0},{"x":540.0,"y":2012.0},{"x":484.0,"y":2012.0}]},"inferText":"전의","inferConfidence":0.9997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":536.0,"y":1975.0},{"x":596.0,"y":1975.0},{"x":596.0,"y":2017.0},{"x":536.0,"y":2017.0}]},"inferText":"여권,","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":163.0,"y":2017.0},{"x":284.0,"y":2017.0},{"x":284.0,"y":2054.0},{"x":163.0,"y":2054.0}]},"inferText":"주민등록증","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":275.0,"y":2012.0},{"x":442.0,"y":2012.0},{"x":442.0,"y":2054.0},{"x":275.0,"y":2054.0}]},"inferText":"발급신청확인서,","inferConfidence":0.9978,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":442.0,"y":2017.0},{"x":517.0,"y":2017.0},{"x":517.0,"y":2054.0},{"x":442.0,"y":2054.0}]},"inferText":"모바일","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":512.0,"y":2017.0},{"x":587.0,"y":2017.0},{"x":587.0,"y":2054.0},{"x":512.0,"y":2054.0}]},"inferText":"신분증","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":168.0,"y":2119.0},{"x":293.0,"y":2119.0},{"x":293.0,"y":2161.0},{"x":168.0,"y":2161.0}]},"inferText":"주민등록증","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":289.0,"y":2124.0},{"x":582.0,"y":2124.0},{"x":582.0,"y":2161.0},{"x":289.0,"y":2161.0}]},"inferText":"미발급자(초,중,고학생포함)","inferConfidence":0.967,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":112.0,"y":2180.0},{"x":219.0,"y":2180.0},{"x":219.0,"y":2226.0},{"x":112.0,"y":2226.0}]},"inferText":"청소년증,","inferConfidence":0.9982,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":210.0,"y":2180.0},{"x":442.0,"y":2180.0},{"x":442.0,"y":2226.0},{"x":210.0,"y":2226.0}]},"inferText":"청소년발급신청확인서,","inferConfidence":0.9951,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":438.0,"y":2184.0},{"x":633.0,"y":2184.0},{"x":633.0,"y":2221.0},{"x":438.0,"y":2221.0}]},"inferText":"기간만료 전의 여권","inferConfidence":0.991,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":238.0,"y":2287.0},{"x":508.0,"y":2287.0},{"x":508.0,"y":2333.0},{"x":238.0,"y":2333.0}]},"inferText":"외국인 (대한민국 국적 외)","inferConfidence":0.9901,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":275.0,"y":2357.0},{"x":428.0,"y":2357.0},{"x":428.0,"y":2394.0},{"x":275.0,"y":2394.0}]},"inferText":"외국인 등록증,","inferConfidence":0.9937,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":419.0,"y":2357.0},{"x":480.0,"y":2357.0},{"x":480.0,"y":2394.0},{"x":419.0,"y":2394.0}]},"inferText":"여권","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":242.0,"y":2478.0},{"x":391.0,"y":2478.0},{"x":391.0,"y":2524.0},{"x":242.0,"y":2524.0}]},"inferText":"인정되지 않는","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":382.0,"y":2478.0},{"x":508.0,"y":2478.0},{"x":508.0,"y":2524.0},{"x":382.0,"y":2524.0}]},"inferText":"무효신분증","inferConfidence":0.9995,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":88.0,"y":2571.0},{"x":168.0,"y":2571.0},{"x":168.0,"y":2608.0},{"x":88.0,"y":2608.0}]},"inferText":"학생증,","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":163.0,"y":2571.0},{"x":247.0,"y":2571.0},{"x":247.0,"y":2608.0},{"x":163.0,"y":2608.0}]},"inferText":"사원증,","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":242.0,"y":2571.0},{"x":405.0,"y":2571.0},{"x":405.0,"y":2608.0},{"x":242.0,"y":2608.0}]},"inferText":"국민건강보험증,","inferConfidence":0.997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":396.0,"y":2571.0},{"x":545.0,"y":2571.0},{"x":545.0,"y":2608.0},{"x":396.0,"y":2608.0}]},"inferText":"주민등록등본,","inferConfidence":0.9991,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":540.0,"y":2566.0},{"x":661.0,"y":2566.0},{"x":661.0,"y":2608.0},{"x":540.0,"y":2608.0}]},"inferText":"의료보험증,","inferConfidence":0.9995,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":79.0,"y":2603.0},{"x":186.0,"y":2603.0},{"x":186.0,"y":2650.0},{"x":79.0,"y":2650.0}]},"inferText":"공무원증,","inferConfidence":0.9985,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":182.0,"y":2608.0},{"x":256.0,"y":2608.0},{"x":256.0,"y":2645.0},{"x":182.0,"y":2645.0}]},"inferText":"기간이","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":251.0,"y":2608.0},{"x":326.0,"y":2608.0},{"x":326.0,"y":2645.0},{"x":251.0,"y":2645.0}]},"inferText":"만료된","inferConfidence":0.9996,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":317.0,"y":2608.0},{"x":382.0,"y":2608.0},{"x":382.0,"y":2645.0},{"x":317.0,"y":2645.0}]},"inferText":"여권,","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":373.0,"y":2608.0},{"x":522.0,"y":2608.0},{"x":522.0,"y":2645.0},{"x":373.0,"y":2645.0}]},"inferText":"기간이 만료된","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":512.0,"y":2608.0},{"x":591.0,"y":2608.0},{"x":591.0,"y":2645.0},{"x":512.0,"y":2645.0}]},"inferText":"외국인","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":587.0,"y":2608.0},{"x":671.0,"y":2608.0},{"x":671.0,"y":2650.0},{"x":587.0,"y":2650.0}]},"inferText":"등록증,","inferConfidence":0.9999,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":75.0,"y":2650.0},{"x":177.0,"y":2650.0},{"x":177.0,"y":2687.0},{"x":75.0,"y":2687.0}]},"inferText":"기관장의","inferConfidence":0.9947,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":172.0,"y":2650.0},{"x":228.0,"y":2650.0},{"x":228.0,"y":2683.0},{"x":172.0,"y":2683.0}]},"inferText":"관인","inferConfidence":0.9994,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":224.0,"y":2650.0},{"x":251.0,"y":2650.0},{"x":251.0,"y":2683.0},{"x":224.0,"y":2683.0}]},"inferText":"및","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":247.0,"y":2650.0},{"x":303.0,"y":2650.0},{"x":303.0,"y":2687.0},{"x":247.0,"y":2687.0}]},"inferText":"투명","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":298.0,"y":2650.0},{"x":484.0,"y":2650.0},{"x":484.0,"y":2687.0},{"x":298.0,"y":2687.0}]},"inferText":"스티커가 부착되어","inferConfidence":0.9614,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":475.0,"y":2650.0},{"x":531.0,"y":2650.0},{"x":531.0,"y":2687.0},{"x":475.0,"y":2687.0}]},"inferText":"있지","inferConfidence":0.9944,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":526.0,"y":2650.0},{"x":582.0,"y":2650.0},{"x":582.0,"y":2687.0},{"x":526.0,"y":2687.0}]},"inferText":"않은","inferConfidence":0.9997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":577.0,"y":2650.0},{"x":675.0,"y":2650.0},{"x":675.0,"y":2687.0},{"x":577.0,"y":2687.0}]},"inferText":"청소년증","inferConfidence":0.9994,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":93.0,"y":2687.0},{"x":149.0,"y":2687.0},{"x":149.0,"y":2729.0},{"x":93.0,"y":2729.0}]},"inferText":"발급","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":140.0,"y":2683.0},{"x":261.0,"y":2683.0},{"x":261.0,"y":2729.0},{"x":140.0,"y":2729.0}]},"inferText":"신청확인서,","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":261.0,"y":2687.0},{"x":317.0,"y":2687.0},{"x":317.0,"y":2729.0},{"x":261.0,"y":2729.0}]},"inferText":"캡처","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":307.0,"y":2692.0},{"x":340.0,"y":2692.0},{"x":340.0,"y":2720.0},{"x":307.0,"y":2720.0}]},"inferText":"된","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":335.0,"y":2683.0},{"x":414.0,"y":2683.0},{"x":414.0,"y":2729.0},{"x":335.0,"y":2729.0}]},"inferText":"모바일","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":405.0,"y":2683.0},{"x":484.0,"y":2683.0},{"x":484.0,"y":2729.0},{"x":405.0,"y":2729.0}]},"inferText":"신분증","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":480.0,"y":2687.0},{"x":508.0,"y":2687.0},{"x":508.0,"y":2720.0},{"x":480.0,"y":2720.0}]},"inferText":"등","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":508.0,"y":2687.0},{"x":587.0,"y":2687.0},{"x":587.0,"y":2729.0},{"x":508.0,"y":2729.0}]},"inferText":"위 유효","inferConfidence":0.9997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":577.0,"y":2683.0},{"x":661.0,"y":2683.0},{"x":661.0,"y":2729.0},{"x":577.0,"y":2729.0}]},"inferText":"신분증","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":228.0,"y":2729.0},{"x":307.0,"y":2729.0},{"x":307.0,"y":2766.0},{"x":228.0,"y":2766.0}]},"inferText":"조건에","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":298.0,"y":2729.0},{"x":354.0,"y":2729.0},{"x":354.0,"y":2766.0},{"x":298.0,"y":2766.0}]},"inferText":"맞지","inferConfidence":0.998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":345.0,"y":2729.0},{"x":405.0,"y":2729.0},{"x":405.0,"y":2766.0},{"x":345.0,"y":2766.0}]},"inferText":"않는","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":396.0,"y":2729.0},{"x":452.0,"y":2729.0},{"x":452.0,"y":2766.0},{"x":396.0,"y":2766.0}]},"inferText":"모든","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":447.0,"y":2729.0},{"x":526.0,"y":2729.0},{"x":526.0,"y":2766.0},{"x":447.0,"y":2766.0}]},"inferText":"신분증","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":2832.0},{"x":140.0,"y":2832.0},{"x":140.0,"y":2869.0},{"x":61.0,"y":2869.0}]},"inferText":"글로벌","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":130.0,"y":2832.0},{"x":251.0,"y":2832.0},{"x":251.0,"y":2869.0},{"x":130.0,"y":2869.0}]},"inferText":"페이지에서","inferConfidence":0.9997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":242.0,"y":2836.0},{"x":298.0,"y":2836.0},{"x":298.0,"y":2869.0},{"x":242.0,"y":2869.0}]},"inferText":"예매","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":298.0,"y":2841.0},{"x":326.0,"y":2841.0},{"x":326.0,"y":2864.0},{"x":298.0,"y":2864.0}]},"inferText":"시,","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":326.0,"y":2832.0},{"x":452.0,"y":2832.0},{"x":452.0,"y":2869.0},{"x":326.0,"y":2869.0}]},"inferText":"오타나 잘못","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":447.0,"y":2832.0},{"x":522.0,"y":2832.0},{"x":522.0,"y":2869.0},{"x":447.0,"y":2869.0}]},"inferText":"기재된","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":512.0,"y":2827.0},{"x":647.0,"y":2827.0},{"x":647.0,"y":2873.0},{"x":512.0,"y":2873.0}]},"inferText":"예매자명 및","inferConfidence":0.9999,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":56.0,"y":2869.0},{"x":182.0,"y":2869.0},{"x":182.0,"y":2911.0},{"x":56.0,"y":2911.0}]},"inferText":"생년월일은","inferConfidence":0.9921,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":172.0,"y":2873.0},{"x":251.0,"y":2873.0},{"x":251.0,"y":2911.0},{"x":172.0,"y":2911.0}]},"inferText":"수정이","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":242.0,"y":2869.0},{"x":368.0,"y":2869.0},{"x":368.0,"y":2911.0},{"x":242.0,"y":2911.0}]},"inferText":"불가능하며,","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":368.0,"y":2873.0},{"x":470.0,"y":2873.0},{"x":470.0,"y":2911.0},{"x":368.0,"y":2911.0}]},"inferText":"공연당일","inferConfidence":0.888,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":461.0,"y":2873.0},{"x":615.0,"y":2873.0},{"x":615.0,"y":2911.0},{"x":461.0,"y":2911.0}]},"inferText":"신분증과 NOL","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":610.0,"y":2873.0},{"x":666.0,"y":2873.0},{"x":666.0,"y":2911.0},{"x":610.0,"y":2911.0}]},"inferText":"티켓","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":2911.0},{"x":210.0,"y":2911.0},{"x":210.0,"y":2948.0},{"x":61.0,"y":2948.0}]},"inferText":"예매 내역서의","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":200.0,"y":2911.0},{"x":279.0,"y":2911.0},{"x":279.0,"y":2953.0},{"x":200.0,"y":2953.0}]},"inferText":"정보가","inferConfidence":0.9986,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":270.0,"y":2911.0},{"x":349.0,"y":2911.0},{"x":349.0,"y":2953.0},{"x":270.0,"y":2953.0}]},"inferText":"정확히","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":345.0,"y":2911.0},{"x":442.0,"y":2911.0},{"x":442.0,"y":2953.0},{"x":345.0,"y":2953.0}]},"inferText":"일치해야","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":433.0,"y":2911.0},{"x":489.0,"y":2911.0},{"x":489.0,"y":2953.0},{"x":433.0,"y":2953.0}]},"inferText":"본인","inferConfidence":0.996,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":484.0,"y":2911.0},{"x":568.0,"y":2911.0},{"x":568.0,"y":2953.0},{"x":484.0,"y":2953.0}]},"inferText":"확인 및","inferConfidence":0.9994,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":559.0,"y":2911.0},{"x":615.0,"y":2911.0},{"x":615.0,"y":2953.0},{"x":559.0,"y":2953.0}]},"inferText":"공연","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":610.0,"y":2911.0},{"x":689.0,"y":2911.0},{"x":689.0,"y":2953.0},{"x":610.0,"y":2953.0}]},"inferText":"관람이","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":2953.0},{"x":186.0,"y":2953.0},{"x":186.0,"y":2990.0},{"x":61.0,"y":2990.0}]},"inferText":"가능합니다.","inferConfidence":0.998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":177.0,"y":2953.0},{"x":238.0,"y":2953.0},{"x":238.0,"y":2990.0},{"x":177.0,"y":2990.0}]},"inferText":"예매","inferConfidence":0.9976,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":228.0,"y":2953.0},{"x":261.0,"y":2953.0},{"x":261.0,"y":2985.0},{"x":228.0,"y":2985.0}]},"inferText":"시","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":256.0,"y":2953.0},{"x":289.0,"y":2953.0},{"x":289.0,"y":2985.0},{"x":256.0,"y":2985.0}]},"inferText":"이","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":284.0,"y":2953.0},{"x":340.0,"y":2953.0},{"x":340.0,"y":2990.0},{"x":284.0,"y":2990.0}]},"inferText":"점을","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":335.0,"y":2953.0},{"x":410.0,"y":2953.0},{"x":410.0,"y":2990.0},{"x":335.0,"y":2990.0}]},"inferText":"반드시","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":405.0,"y":2953.0},{"x":550.0,"y":2953.0},{"x":550.0,"y":2990.0},{"x":405.0,"y":2990.0}]},"inferText":"유의해 주시기","inferConfidence":0.9977,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":545.0,"y":2953.0},{"x":647.0,"y":2953.0},{"x":647.0,"y":2990.0},{"x":545.0,"y":2990.0}]},"inferText":"바랍니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3013.0},{"x":238.0,"y":3013.0},{"x":238.0,"y":3055.0},{"x":61.0,"y":3055.0}]},"inferText":"개명 전 이름으로","inferConfidence":0.9996,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":228.0,"y":3018.0},{"x":331.0,"y":3018.0},{"x":331.0,"y":3055.0},{"x":228.0,"y":3055.0}]},"inferText":"예매하신","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":326.0,"y":3018.0},{"x":382.0,"y":3018.0},{"x":382.0,"y":3055.0},{"x":326.0,"y":3055.0}]},"inferText":"분은","inferConfidence":0.9997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":373.0,"y":3018.0},{"x":428.0,"y":3018.0},{"x":428.0,"y":3055.0},{"x":373.0,"y":3055.0}]},"inferText":"개명","inferConfidence":0.9996,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":424.0,"y":3018.0},{"x":452.0,"y":3018.0},{"x":452.0,"y":3050.0},{"x":424.0,"y":3050.0}]},"inferText":"전","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":447.0,"y":3018.0},{"x":531.0,"y":3018.0},{"x":531.0,"y":3055.0},{"x":447.0,"y":3055.0}]},"inferText":"신분증,","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":526.0,"y":3013.0},{"x":624.0,"y":3013.0},{"x":624.0,"y":3055.0},{"x":526.0,"y":3055.0}]},"inferText":"기관장의","inferConfidence":0.9985,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":619.0,"y":3013.0},{"x":694.0,"y":3013.0},{"x":694.0,"y":3050.0},{"x":619.0,"y":3050.0}]},"inferText":"관인이","inferConfidence":0.9992,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3055.0},{"x":116.0,"y":3055.0},{"x":116.0,"y":3092.0},{"x":61.0,"y":3092.0}]},"inferText":"있는","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":112.0,"y":3055.0},{"x":168.0,"y":3055.0},{"x":168.0,"y":3092.0},{"x":112.0,"y":3092.0}]},"inferText":"개명","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":158.0,"y":3055.0},{"x":242.0,"y":3055.0},{"x":242.0,"y":3092.0},{"x":158.0,"y":3092.0}]},"inferText":"확인서,","inferConfidence":0.999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":238.0,"y":3055.0},{"x":410.0,"y":3055.0},{"x":410.0,"y":3092.0},{"x":238.0,"y":3092.0}]},"inferText":"주민등록초본 등","inferConfidence":0.9925,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":401.0,"y":3055.0},{"x":475.0,"y":3055.0},{"x":475.0,"y":3092.0},{"x":401.0,"y":3092.0}]},"inferText":"개명에","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":470.0,"y":3055.0},{"x":522.0,"y":3055.0},{"x":522.0,"y":3088.0},{"x":470.0,"y":3088.0}]},"inferText":"의한","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":517.0,"y":3055.0},{"x":573.0,"y":3055.0},{"x":573.0,"y":3092.0},{"x":517.0,"y":3092.0}]},"inferText":"이름","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":568.0,"y":3055.0},{"x":643.0,"y":3055.0},{"x":643.0,"y":3092.0},{"x":568.0,"y":3092.0}]},"inferText":"변경을","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":70.0,"y":3097.0},{"x":144.0,"y":3097.0},{"x":144.0,"y":3134.0},{"x":70.0,"y":3134.0}]},"inferText":"증명할","inferConfidence":0.9993,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":144.0,"y":3102.0},{"x":168.0,"y":3102.0},{"x":168.0,"y":3130.0},{"x":144.0,"y":3130.0}]},"inferText":"수","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":163.0,"y":3097.0},{"x":224.0,"y":3097.0},{"x":224.0,"y":3134.0},{"x":163.0,"y":3134.0}]},"inferText":"있는","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":214.0,"y":3097.0},{"x":293.0,"y":3097.0},{"x":293.0,"y":3134.0},{"x":214.0,"y":3134.0}]},"inferText":"서류를","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":284.0,"y":3097.0},{"x":340.0,"y":3097.0},{"x":340.0,"y":3134.0},{"x":284.0,"y":3134.0}]},"inferText":"지참","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":335.0,"y":3097.0},{"x":438.0,"y":3097.0},{"x":438.0,"y":3134.0},{"x":335.0,"y":3134.0}]},"inferText":"바랍니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":65.0,"y":3158.0},{"x":140.0,"y":3158.0},{"x":140.0,"y":3195.0},{"x":65.0,"y":3195.0}]},"inferText":"모바일","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":135.0,"y":3158.0},{"x":233.0,"y":3158.0},{"x":233.0,"y":3195.0},{"x":135.0,"y":3195.0}]},"inferText":"신분증은","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":224.0,"y":3158.0},{"x":279.0,"y":3158.0},{"x":279.0,"y":3195.0},{"x":224.0,"y":3195.0}]},"inferText":"전용","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":275.0,"y":3158.0},{"x":303.0,"y":3158.0},{"x":303.0,"y":3190.0},{"x":275.0,"y":3190.0}]},"inferText":"앱","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":303.0,"y":3158.0},{"x":354.0,"y":3158.0},{"x":354.0,"y":3190.0},{"x":303.0,"y":3190.0}]},"inferText":"외의","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":354.0,"y":3158.0},{"x":461.0,"y":3158.0},{"x":461.0,"y":3199.0},{"x":354.0,"y":3199.0}]},"inferText":"방법(양도,","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":456.0,"y":3158.0},{"x":517.0,"y":3158.0},{"x":517.0,"y":3199.0},{"x":456.0,"y":3199.0}]},"inferText":"캡처,","inferConfidence":0.9979,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":508.0,"y":3158.0},{"x":568.0,"y":3158.0},{"x":568.0,"y":3195.0},{"x":508.0,"y":3195.0}]},"inferText":"사진","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":564.0,"y":3158.0},{"x":666.0,"y":3158.0},{"x":666.0,"y":3195.0},{"x":564.0,"y":3195.0}]},"inferText":"등)으로는","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":56.0,"y":3195.0},{"x":158.0,"y":3195.0},{"x":158.0,"y":3237.0},{"x":56.0,"y":3237.0}]},"inferText":"인정되지","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":154.0,"y":3199.0},{"x":251.0,"y":3199.0},{"x":251.0,"y":3237.0},{"x":154.0,"y":3237.0}]},"inferText":"않으므로","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":247.0,"y":3199.0},{"x":363.0,"y":3199.0},{"x":363.0,"y":3237.0},{"x":247.0,"y":3237.0}]},"inferText":"유의하시기","inferConfidence":0.9976,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":354.0,"y":3199.0},{"x":461.0,"y":3199.0},{"x":461.0,"y":3237.0},{"x":354.0,"y":3237.0}]},"inferText":"바랍니다.","inferConfidence":0.9996,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3260.0},{"x":182.0,"y":3260.0},{"x":182.0,"y":3297.0},{"x":61.0,"y":3297.0}]},"inferText":"청소년증은","inferConfidence":0.9993,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":177.0,"y":3260.0},{"x":270.0,"y":3260.0},{"x":270.0,"y":3297.0},{"x":177.0,"y":3297.0}]},"inferText":"주소지와","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":270.0,"y":3260.0},{"x":363.0,"y":3260.0},{"x":363.0,"y":3297.0},{"x":270.0,"y":3297.0}]},"inferText":"관계없이","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":354.0,"y":3260.0},{"x":433.0,"y":3260.0},{"x":433.0,"y":3297.0},{"x":354.0,"y":3297.0}]},"inferText":"가까운","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":428.0,"y":3260.0},{"x":568.0,"y":3260.0},{"x":568.0,"y":3297.0},{"x":428.0,"y":3297.0}]},"inferText":"주민센터에서","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":559.0,"y":3260.0},{"x":619.0,"y":3260.0},{"x":619.0,"y":3297.0},{"x":559.0,"y":3297.0}]},"inferText":"본인","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":610.0,"y":3260.0},{"x":666.0,"y":3260.0},{"x":666.0,"y":3297.0},{"x":610.0,"y":3297.0}]},"inferText":"또는","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3302.0},{"x":158.0,"y":3302.0},{"x":158.0,"y":3339.0},{"x":61.0,"y":3339.0}]},"inferText":"대리인이","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":154.0,"y":3302.0},{"x":233.0,"y":3302.0},{"x":233.0,"y":3339.0},{"x":154.0,"y":3339.0}]},"inferText":"신청할","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":228.0,"y":3307.0},{"x":247.0,"y":3307.0},{"x":247.0,"y":3330.0},{"x":228.0,"y":3330.0}]},"inferText":"수","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":247.0,"y":3302.0},{"x":354.0,"y":3302.0},{"x":354.0,"y":3339.0},{"x":247.0,"y":3339.0}]},"inferText":"있습니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":65.0,"y":3362.0},{"x":158.0,"y":3362.0},{"x":158.0,"y":3400.0},{"x":65.0,"y":3400.0}]},"inferText":"청소년증","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":154.0,"y":3362.0},{"x":256.0,"y":3362.0},{"x":256.0,"y":3404.0},{"x":154.0,"y":3404.0}]},"inferText":"발급 신청","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":247.0,"y":3358.0},{"x":419.0,"y":3358.0},{"x":419.0,"y":3404.0},{"x":247.0,"y":3404.0}]},"inferText":"확인서는 반드시","inferConfidence":0.9995,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":414.0,"y":3362.0},{"x":512.0,"y":3362.0},{"x":512.0,"y":3404.0},{"x":414.0,"y":3404.0}]},"inferText":"기관장의","inferConfidence":0.9952,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":508.0,"y":3367.0},{"x":582.0,"y":3367.0},{"x":582.0,"y":3400.0},{"x":508.0,"y":3400.0}]},"inferText":"관인이","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":577.0,"y":3362.0},{"x":703.0,"y":3362.0},{"x":703.0,"y":3400.0},{"x":577.0,"y":3400.0}]},"inferText":"있어야 하며,","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3404.0},{"x":121.0,"y":3404.0},{"x":121.0,"y":3442.0},{"x":61.0,"y":3442.0}]},"inferText":"사진,","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":116.0,"y":3404.0},{"x":224.0,"y":3404.0},{"x":224.0,"y":3442.0},{"x":116.0,"y":3442.0}]},"inferText":"인적 사항,","inferConfidence":0.9723,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":224.0,"y":3404.0},{"x":275.0,"y":3404.0},{"x":275.0,"y":3442.0},{"x":224.0,"y":3442.0}]},"inferText":"발급","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":270.0,"y":3404.0},{"x":326.0,"y":3404.0},{"x":326.0,"y":3442.0},{"x":270.0,"y":3442.0}]},"inferText":"번호,","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":326.0,"y":3404.0},{"x":470.0,"y":3404.0},{"x":470.0,"y":3442.0},{"x":326.0,"y":3442.0}]},"inferText":"유효기간 등을","inferConfidence":0.9995,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":461.0,"y":3400.0},{"x":568.0,"y":3400.0},{"x":568.0,"y":3446.0},{"x":461.0,"y":3446.0}]},"inferText":"포함하고,","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":564.0,"y":3404.0},{"x":708.0,"y":3404.0},{"x":708.0,"y":3442.0},{"x":564.0,"y":3442.0}]},"inferText":"투명 스티커가","inferConfidence":0.9999,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3442.0},{"x":140.0,"y":3442.0},{"x":140.0,"y":3484.0},{"x":61.0,"y":3484.0}]},"inferText":"부착된","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":135.0,"y":3442.0},{"x":233.0,"y":3442.0},{"x":233.0,"y":3484.0},{"x":135.0,"y":3484.0}]},"inferText":"경우에만","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":224.0,"y":3442.0},{"x":279.0,"y":3442.0},{"x":279.0,"y":3484.0},{"x":224.0,"y":3484.0}]},"inferText":"유효","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":275.0,"y":3442.0},{"x":391.0,"y":3442.0},{"x":391.0,"y":3484.0},{"x":275.0,"y":3484.0}]},"inferText":"신분증으로","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":387.0,"y":3442.0},{"x":508.0,"y":3442.0},{"x":508.0,"y":3474.0},{"x":387.0,"y":3474.0}]},"inferText":"인정됩니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":503.0,"y":3442.0},{"x":577.0,"y":3442.0},{"x":577.0,"y":3479.0},{"x":503.0,"y":3479.0}]},"inferText":"변조가","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":573.0,"y":3442.0},{"x":671.0,"y":3442.0},{"x":671.0,"y":3484.0},{"x":573.0,"y":3484.0}]},"inferText":"의심되는","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":661.0,"y":3442.0},{"x":717.0,"y":3442.0},{"x":717.0,"y":3474.0},{"x":661.0,"y":3474.0}]},"inferText":"경우","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3484.0},{"x":158.0,"y":3484.0},{"x":158.0,"y":3521.0},{"x":61.0,"y":3521.0}]},"inferText":"인정되지","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":154.0,"y":3484.0},{"x":228.0,"y":3484.0},{"x":228.0,"y":3521.0},{"x":154.0,"y":3521.0}]},"inferText":"않으니","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":224.0,"y":3484.0},{"x":340.0,"y":3484.0},{"x":340.0,"y":3521.0},{"x":224.0,"y":3521.0}]},"inferText":"유의하시기","inferConfidence":0.9989,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":335.0,"y":3484.0},{"x":438.0,"y":3484.0},{"x":438.0,"y":3521.0},{"x":335.0,"y":3521.0}]},"inferText":"바랍니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3549.0},{"x":121.0,"y":3549.0},{"x":121.0,"y":3586.0},{"x":61.0,"y":3586.0}]},"inferText":"가족","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":112.0,"y":3544.0},{"x":186.0,"y":3544.0},{"x":186.0,"y":3586.0},{"x":112.0,"y":3586.0}]},"inferText":"명의로","inferConfidence":0.9995,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":182.0,"y":3549.0},{"x":261.0,"y":3549.0},{"x":261.0,"y":3586.0},{"x":182.0,"y":3586.0}]},"inferText":"예매한","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":251.0,"y":3549.0},{"x":289.0,"y":3549.0},{"x":289.0,"y":3586.0},{"x":251.0,"y":3586.0}]},"inferText":"만","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":284.0,"y":3549.0},{"x":387.0,"y":3549.0},{"x":387.0,"y":3586.0},{"x":284.0,"y":3586.0}]},"inferText":"14세미만","inferConfidence":0.91,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":382.0,"y":3549.0},{"x":480.0,"y":3549.0},{"x":480.0,"y":3581.0},{"x":382.0,"y":3581.0}]},"inferText":"미성년자","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":470.0,"y":3544.0},{"x":624.0,"y":3544.0},{"x":624.0,"y":3586.0},{"x":470.0,"y":3586.0}]},"inferText":"예매자는 티켓","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":615.0,"y":3549.0},{"x":699.0,"y":3549.0},{"x":699.0,"y":3586.0},{"x":615.0,"y":3586.0}]},"inferText":"수령 시","inferConfidence":0.9989,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":65.0,"y":3586.0},{"x":168.0,"y":3586.0},{"x":168.0,"y":3623.0},{"x":65.0,"y":3623.0}]},"inferText":"'예매자의","inferConfidence":0.9976,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":163.0,"y":3586.0},{"x":219.0,"y":3586.0},{"x":219.0,"y":3623.0},{"x":163.0,"y":3623.0}]},"inferText":"실물","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":205.0,"y":3581.0},{"x":303.0,"y":3581.0},{"x":303.0,"y":3628.0},{"x":205.0,"y":3628.0}]},"inferText":"신분증',","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":293.0,"y":3586.0},{"x":354.0,"y":3586.0},{"x":354.0,"y":3623.0},{"x":293.0,"y":3623.0}]},"inferText":"'예매","inferConfidence":0.9963,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":340.0,"y":3581.0},{"x":438.0,"y":3581.0},{"x":438.0,"y":3628.0},{"x":340.0,"y":3628.0}]},"inferText":"내역서',","inferConfidence":0.992,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":433.0,"y":3586.0},{"x":531.0,"y":3586.0},{"x":531.0,"y":3623.0},{"x":433.0,"y":3623.0}]},"inferText":"'수령인의","inferConfidence":0.9874,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":526.0,"y":3586.0},{"x":582.0,"y":3586.0},{"x":582.0,"y":3623.0},{"x":526.0,"y":3623.0}]},"inferText":"실물","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":573.0,"y":3586.0},{"x":661.0,"y":3586.0},{"x":661.0,"y":3623.0},{"x":573.0,"y":3623.0}]},"inferText":"신분증',","inferConfidence":0.9996,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3628.0},{"x":168.0,"y":3628.0},{"x":168.0,"y":3665.0},{"x":61.0,"y":3665.0}]},"inferText":"'예매자와","inferConfidence":0.9986,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":163.0,"y":3628.0},{"x":261.0,"y":3628.0},{"x":261.0,"y":3665.0},{"x":163.0,"y":3665.0}]},"inferText":"수령인의","inferConfidence":0.9984,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":251.0,"y":3628.0},{"x":377.0,"y":3628.0},{"x":377.0,"y":3665.0},{"x":251.0,"y":3665.0}]},"inferText":"가족관계를","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":368.0,"y":3628.0},{"x":447.0,"y":3628.0},{"x":447.0,"y":3665.0},{"x":368.0,"y":3665.0}]},"inferText":"증명할","inferConfidence":0.9996,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":438.0,"y":3628.0},{"x":522.0,"y":3628.0},{"x":522.0,"y":3665.0},{"x":438.0,"y":3665.0}]},"inferText":"수 있는","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":512.0,"y":3628.0},{"x":568.0,"y":3628.0},{"x":568.0,"y":3665.0},{"x":512.0,"y":3665.0}]},"inferText":"원본","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":559.0,"y":3628.0},{"x":619.0,"y":3628.0},{"x":619.0,"y":3665.0},{"x":559.0,"y":3665.0}]},"inferText":"서류","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":56.0,"y":3661.0},{"x":219.0,"y":3661.0},{"x":219.0,"y":3707.0},{"x":56.0,"y":3707.0}]},"inferText":"(주민등록등본,","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":214.0,"y":3665.0},{"x":340.0,"y":3665.0},{"x":340.0,"y":3707.0},{"x":214.0,"y":3707.0}]},"inferText":"의료보험증,","inferConfidence":0.9986,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":331.0,"y":3665.0},{"x":577.0,"y":3665.0},{"x":577.0,"y":3707.0},{"x":331.0,"y":3707.0}]},"inferText":"가족관계증명서)'를 모두","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":568.0,"y":3661.0},{"x":675.0,"y":3661.0},{"x":675.0,"y":3707.0},{"x":568.0,"y":3707.0}]},"inferText":"지참해야","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3707.0},{"x":144.0,"y":3707.0},{"x":144.0,"y":3744.0},{"x":61.0,"y":3744.0}]},"inferText":"합니다.","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":135.0,"y":3707.0},{"x":205.0,"y":3707.0},{"x":205.0,"y":3744.0},{"x":135.0,"y":3744.0}]},"inferText":"(가족","inferConfidence":0.9995,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":196.0,"y":3707.0},{"x":298.0,"y":3707.0},{"x":298.0,"y":3744.0},{"x":196.0,"y":3744.0}]},"inferText":"명의 ID에","inferConfidence":0.9984,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":293.0,"y":3707.0},{"x":349.0,"y":3707.0},{"x":349.0,"y":3744.0},{"x":293.0,"y":3744.0}]},"inferText":"대한","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":345.0,"y":3707.0},{"x":470.0,"y":3707.0},{"x":470.0,"y":3744.0},{"x":345.0,"y":3744.0}]},"inferText":"안내는 일반","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":461.0,"y":3707.0},{"x":559.0,"y":3707.0},{"x":559.0,"y":3744.0},{"x":461.0,"y":3744.0}]},"inferText":"예매에만","inferConfidence":0.9997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":554.0,"y":3707.0},{"x":661.0,"y":3707.0},{"x":661.0,"y":3744.0},{"x":554.0,"y":3744.0}]},"inferText":"해당되며,","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3744.0},{"x":186.0,"y":3744.0},{"x":186.0,"y":3786.0},{"x":61.0,"y":3786.0}]},"inferText":"가족 관계가","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":182.0,"y":3749.0},{"x":307.0,"y":3749.0},{"x":307.0,"y":3786.0},{"x":182.0,"y":3786.0}]},"inferText":"아닌 타인의","inferConfidence":0.9995,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":303.0,"y":3749.0},{"x":354.0,"y":3749.0},{"x":354.0,"y":3786.0},{"x":303.0,"y":3786.0}]},"inferText":"대리","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":349.0,"y":3744.0},{"x":428.0,"y":3744.0},{"x":428.0,"y":3786.0},{"x":349.0,"y":3786.0}]},"inferText":"수령은","inferConfidence":0.9997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":424.0,"y":3744.0},{"x":554.0,"y":3744.0},{"x":554.0,"y":3786.0},{"x":424.0,"y":3786.0}]},"inferText":"불가합니다.)","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3814.0},{"x":140.0,"y":3814.0},{"x":140.0,"y":3851.0},{"x":61.0,"y":3851.0}]},"inferText":"지정된","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":130.0,"y":3814.0},{"x":186.0,"y":3814.0},{"x":186.0,"y":3851.0},{"x":130.0,"y":3851.0}]},"inferText":"실물","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":182.0,"y":3814.0},{"x":256.0,"y":3814.0},{"x":256.0,"y":3851.0},{"x":182.0,"y":3851.0}]},"inferText":"신분증","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":251.0,"y":3814.0},{"x":303.0,"y":3814.0},{"x":303.0,"y":3842.0},{"x":251.0,"y":3842.0}]},"inferText":"외의","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":298.0,"y":3814.0},{"x":396.0,"y":3814.0},{"x":396.0,"y":3847.0},{"x":298.0,"y":3847.0}]},"inferText":"미지참에","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":391.0,"y":3814.0},{"x":447.0,"y":3814.0},{"x":447.0,"y":3851.0},{"x":391.0,"y":3851.0}]},"inferText":"대한","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":442.0,"y":3814.0},{"x":517.0,"y":3814.0},{"x":517.0,"y":3851.0},{"x":442.0,"y":3851.0}]},"inferText":"책임은","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":512.0,"y":3814.0},{"x":605.0,"y":3814.0},{"x":605.0,"y":3847.0},{"x":512.0,"y":3847.0}]},"inferText":"본인에게","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":601.0,"y":3814.0},{"x":685.0,"y":3814.0},{"x":685.0,"y":3847.0},{"x":601.0,"y":3847.0}]},"inferText":"있으며,","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3851.0},{"x":116.0,"y":3851.0},{"x":116.0,"y":3889.0},{"x":61.0,"y":3889.0}]},"inferText":"티켓","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":112.0,"y":3851.0},{"x":214.0,"y":3851.0},{"x":214.0,"y":3889.0},{"x":112.0,"y":3889.0}]},"inferText":"미수령 시","inferConfidence":0.9995,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":210.0,"y":3851.0},{"x":317.0,"y":3851.0},{"x":317.0,"y":3889.0},{"x":210.0,"y":3889.0}]},"inferText":"당일 티켓","inferConfidence":0.9988,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":307.0,"y":3851.0},{"x":387.0,"y":3851.0},{"x":387.0,"y":3884.0},{"x":307.0,"y":3884.0}]},"inferText":"취소 및","inferConfidence":0.9986,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":382.0,"y":3851.0},{"x":461.0,"y":3851.0},{"x":461.0,"y":3889.0},{"x":382.0,"y":3889.0}]},"inferText":"환불은","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":456.0,"y":3851.0},{"x":577.0,"y":3851.0},{"x":577.0,"y":3889.0},{"x":456.0,"y":3889.0}]},"inferText":"불가하므로","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3893.0},{"x":182.0,"y":3893.0},{"x":182.0,"y":3931.0},{"x":61.0,"y":3931.0}]},"inferText":"유의하시기","inferConfidence":0.9992,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":172.0,"y":3889.0},{"x":279.0,"y":3889.0},{"x":279.0,"y":3931.0},{"x":172.0,"y":3931.0}]},"inferText":"바랍니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3954.0},{"x":140.0,"y":3954.0},{"x":140.0,"y":3991.0},{"x":61.0,"y":3991.0}]},"inferText":"타인의","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":135.0,"y":3954.0},{"x":210.0,"y":3954.0},{"x":210.0,"y":3991.0},{"x":135.0,"y":3991.0}]},"inferText":"신분을","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":205.0,"y":3954.0},{"x":279.0,"y":3954.0},{"x":279.0,"y":3991.0},{"x":205.0,"y":3991.0}]},"inferText":"도용해","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":275.0,"y":3954.0},{"x":326.0,"y":3954.0},{"x":326.0,"y":3991.0},{"x":275.0,"y":3991.0}]},"inferText":"티켓","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":321.0,"y":3954.0},{"x":442.0,"y":3954.0},{"x":442.0,"y":3991.0},{"x":321.0,"y":3991.0}]},"inferText":"예매하거나","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":438.0,"y":3954.0},{"x":489.0,"y":3954.0},{"x":489.0,"y":3991.0},{"x":438.0,"y":3991.0}]},"inferText":"본인","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":484.0,"y":3954.0},{"x":559.0,"y":3954.0},{"x":559.0,"y":3991.0},{"x":484.0,"y":3991.0}]},"inferText":"확인을","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":554.0,"y":3954.0},{"x":661.0,"y":3954.0},{"x":661.0,"y":3991.0},{"x":554.0,"y":3991.0}]},"inferText":"하는 일이","inferConfidence":0.9997,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":3996.0},{"x":158.0,"y":3996.0},{"x":158.0,"y":4033.0},{"x":61.0,"y":4033.0}]},"inferText":"발생하고","inferConfidence":0.9994,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":149.0,"y":3996.0},{"x":256.0,"y":3996.0},{"x":256.0,"y":4028.0},{"x":149.0,"y":4028.0}]},"inferText":"있으므로.","inferConfidence":0.9993,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":251.0,"y":3996.0},{"x":331.0,"y":3996.0},{"x":331.0,"y":4033.0},{"x":251.0,"y":4033.0}]},"inferText":"반드시","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":321.0,"y":3996.0},{"x":401.0,"y":3996.0},{"x":401.0,"y":4033.0},{"x":321.0,"y":4033.0}]},"inferText":"본인의","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":391.0,"y":3991.0},{"x":447.0,"y":3991.0},{"x":447.0,"y":4033.0},{"x":391.0,"y":4033.0}]},"inferText":"실물","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":438.0,"y":3991.0},{"x":540.0,"y":3991.0},{"x":540.0,"y":4033.0},{"x":438.0,"y":4033.0}]},"inferText":"신분증을","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":531.0,"y":3996.0},{"x":657.0,"y":3996.0},{"x":657.0,"y":4033.0},{"x":531.0,"y":4033.0}]},"inferText":"지참하시기","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":4033.0},{"x":233.0,"y":4033.0},{"x":233.0,"y":4070.0},{"x":61.0,"y":4070.0}]},"inferText":"바랍니다. 지정된","inferConfidence":0.994,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":228.0,"y":4033.0},{"x":307.0,"y":4033.0},{"x":307.0,"y":4075.0},{"x":228.0,"y":4075.0}]},"inferText":"신분증","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":303.0,"y":4033.0},{"x":428.0,"y":4033.0},{"x":428.0,"y":4070.0},{"x":303.0,"y":4070.0}]},"inferText":"내용은 예매","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":419.0,"y":4033.0},{"x":475.0,"y":4033.0},{"x":475.0,"y":4070.0},{"x":419.0,"y":4070.0}]},"inferText":"전에","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":470.0,"y":4033.0},{"x":498.0,"y":4033.0},{"x":498.0,"y":4066.0},{"x":470.0,"y":4066.0}]},"inferText":"꼭","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":498.0,"y":4033.0},{"x":596.0,"y":4033.0},{"x":596.0,"y":4070.0},{"x":498.0,"y":4070.0}]},"inferText":"확인하고","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":587.0,"y":4033.0},{"x":666.0,"y":4033.0},{"x":666.0,"y":4070.0},{"x":587.0,"y":4070.0}]},"inferText":"준비해","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":4075.0},{"x":140.0,"y":4075.0},{"x":140.0,"y":4112.0},{"x":61.0,"y":4112.0}]},"inferText":"주시기","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":130.0,"y":4075.0},{"x":238.0,"y":4075.0},{"x":238.0,"y":4112.0},{"x":130.0,"y":4112.0}]},"inferText":"바랍니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":4177.0},{"x":168.0,"y":4177.0},{"x":168.0,"y":4219.0},{"x":61.0,"y":4219.0}]},"inferText":"티켓 수령","inferConfidence":0.9923,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":158.0,"y":4177.0},{"x":289.0,"y":4177.0},{"x":289.0,"y":4219.0},{"x":158.0,"y":4219.0}]},"inferText":"후 주의사항","inferConfidence":0.9998,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":4243.0},{"x":116.0,"y":4243.0},{"x":116.0,"y":4280.0},{"x":61.0,"y":4280.0}]},"inferText":"배송","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":112.0,"y":4243.0},{"x":168.0,"y":4243.0},{"x":168.0,"y":4280.0},{"x":112.0,"y":4280.0}]},"inferText":"받은","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":158.0,"y":4243.0},{"x":238.0,"y":4243.0},{"x":238.0,"y":4280.0},{"x":158.0,"y":4280.0}]},"inferText":"티켓을","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":233.0,"y":4243.0},{"x":307.0,"y":4243.0},{"x":307.0,"y":4280.0},{"x":233.0,"y":4280.0}]},"inferText":"포함한","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":303.0,"y":4243.0},{"x":456.0,"y":4243.0},{"x":456.0,"y":4280.0},{"x":303.0,"y":4280.0}]},"inferText":"모든 티켓 수령","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":447.0,"y":4243.0},{"x":522.0,"y":4243.0},{"x":522.0,"y":4280.0},{"x":447.0,"y":4280.0}]},"inferText":"후에는","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":517.0,"y":4243.0},{"x":717.0,"y":4243.0},{"x":717.0,"y":4280.0},{"x":517.0,"y":4280.0}]},"inferText":"아래 예시를 포함한","inferConfidence":0.9999,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":4285.0},{"x":140.0,"y":4285.0},{"x":140.0,"y":4317.0},{"x":61.0,"y":4317.0}]},"inferText":"어떠한","inferConfidence":0.9994,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":130.0,"y":4280.0},{"x":284.0,"y":4280.0},{"x":284.0,"y":4322.0},{"x":130.0,"y":4322.0}]},"inferText":"사유로도(분실,","inferConfidence":0.9991,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":279.0,"y":4280.0},{"x":335.0,"y":4280.0},{"x":335.0,"y":4322.0},{"x":279.0,"y":4322.0}]},"inferText":"도난,","inferConfidence":0.9971,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":335.0,"y":4285.0},{"x":391.0,"y":4285.0},{"x":391.0,"y":4322.0},{"x":335.0,"y":4322.0}]},"inferText":"훼손,","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":387.0,"y":4277.0},{"x":514.0,"y":4285.0},{"x":512.0,"y":4325.0},{"x":384.0,"y":4317.0}]},"inferText":"현장 미소지","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":508.0,"y":4285.0},{"x":545.0,"y":4285.0},{"x":545.0,"y":4317.0},{"x":508.0,"y":4317.0}]},"inferText":"등)","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":540.0,"y":4280.0},{"x":643.0,"y":4280.0},{"x":643.0,"y":4322.0},{"x":540.0,"y":4322.0}]},"inferText":"재발권이","inferConfidence":0.9999,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":56.0,"y":4317.0},{"x":163.0,"y":4317.0},{"x":163.0,"y":4364.0},{"x":56.0,"y":4364.0}]},"inferText":"불가하며","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":154.0,"y":4322.0},{"x":312.0,"y":4322.0},{"x":312.0,"y":4359.0},{"x":154.0,"y":4359.0}]},"inferText":"취소/환불 또한","inferConfidence":0.9985,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":303.0,"y":4322.0},{"x":428.0,"y":4322.0},{"x":428.0,"y":4359.0},{"x":303.0,"y":4359.0}]},"inferText":"불가합니다.","inferConfidence":0.9992,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":419.0,"y":4322.0},{"x":475.0,"y":4322.0},{"x":475.0,"y":4359.0},{"x":419.0,"y":4359.0}]},"inferText":"티켓","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":470.0,"y":4322.0},{"x":550.0,"y":4322.0},{"x":550.0,"y":4359.0},{"x":470.0,"y":4359.0}]},"inferText":"미소지","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":540.0,"y":4322.0},{"x":573.0,"y":4322.0},{"x":573.0,"y":4354.0},{"x":540.0,"y":4354.0}]},"inferText":"시","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":568.0,"y":4322.0},{"x":643.0,"y":4322.0},{"x":643.0,"y":4359.0},{"x":568.0,"y":4359.0}]},"inferText":"공연장","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":638.0,"y":4322.0},{"x":717.0,"y":4322.0},{"x":717.0,"y":4359.0},{"x":638.0,"y":4359.0}]},"inferText":"입장이","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":61.0,"y":4364.0},{"x":168.0,"y":4364.0},{"x":168.0,"y":4401.0},{"x":61.0,"y":4401.0}]},"inferText":"불가하니,","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":158.0,"y":4364.0},{"x":214.0,"y":4364.0},{"x":214.0,"y":4401.0},{"x":158.0,"y":4401.0}]},"inferText":"티켓","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":210.0,"y":4364.0},{"x":284.0,"y":4364.0},{"x":284.0,"y":4401.0},{"x":210.0,"y":4401.0}]},"inferText":"보관에","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":279.0,"y":4364.0},{"x":396.0,"y":4364.0},{"x":396.0,"y":4401.0},{"x":279.0,"y":4401.0}]},"inferText":"유의하시기","inferConfidence":0.9994,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":391.0,"y":4364.0},{"x":447.0,"y":4364.0},{"x":447.0,"y":4401.0},{"x":391.0,"y":4401.0}]},"inferText":"바랍","inferConfidence":0.9997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":438.0,"y":4364.0},{"x":494.0,"y":4364.0},{"x":494.0,"y":4401.0},{"x":438.0,"y":4401.0}]},"inferText":"니다.","inferConfidence":0.9977,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":70.0,"y":4434.0},{"x":93.0,"y":4434.0},{"x":93.0,"y":4462.0},{"x":70.0,"y":4462.0}]},"inferText":"A","inferConfidence":0.9997,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":102.0,"y":4429.0},{"x":196.0,"y":4429.0},{"x":196.0,"y":4466.0},{"x":102.0,"y":4466.0}]},"inferText":"경비실에","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":191.0,"y":4429.0},{"x":247.0,"y":4429.0},{"x":247.0,"y":4466.0},{"x":191.0,"y":4466.0}]},"inferText":"맡겨","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":238.0,"y":4429.0},{"x":335.0,"y":4429.0},{"x":335.0,"y":4466.0},{"x":238.0,"y":4466.0}]},"inferText":"두었는데","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":331.0,"y":4429.0},{"x":387.0,"y":4429.0},{"x":387.0,"y":4466.0},{"x":331.0,"y":4466.0}]},"inferText":"다른","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":377.0,"y":4429.0},{"x":433.0,"y":4429.0},{"x":433.0,"y":4466.0},{"x":377.0,"y":4466.0}]},"inferText":"분이","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":428.0,"y":4429.0},{"x":550.0,"y":4429.0},{"x":550.0,"y":4466.0},{"x":428.0,"y":4466.0}]},"inferText":"찾아갔다고","inferConfidence":0.9981,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":540.0,"y":4429.0},{"x":624.0,"y":4429.0},{"x":624.0,"y":4462.0},{"x":540.0,"y":4462.0}]},"inferText":"합니다.","inferConfidence":0.9995,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":70.0,"y":4499.0},{"x":93.0,"y":4499.0},{"x":93.0,"y":4527.0},{"x":70.0,"y":4527.0}]},"inferText":"B","inferConfidence":0.991,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":4494.0},{"x":154.0,"y":4494.0},{"x":154.0,"y":4531.0},{"x":98.0,"y":4531.0}]},"inferText":"회사","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":149.0,"y":4494.0},{"x":251.0,"y":4494.0},{"x":251.0,"y":4531.0},{"x":149.0,"y":4531.0}]},"inferText":"동료(혹은","inferConfidence":0.9987,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":247.0,"y":4494.0},{"x":331.0,"y":4494.0},{"x":331.0,"y":4531.0},{"x":247.0,"y":4531.0}]},"inferText":"가족)가","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":326.0,"y":4494.0},{"x":382.0,"y":4494.0},{"x":382.0,"y":4531.0},{"x":326.0,"y":4531.0}]},"inferText":"대신","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":373.0,"y":4494.0},{"x":587.0,"y":4494.0},{"x":587.0,"y":4531.0},{"x":373.0,"y":4531.0}]},"inferText":"수령하였는데 어디에","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":577.0,"y":4494.0},{"x":675.0,"y":4494.0},{"x":675.0,"y":4531.0},{"x":577.0,"y":4531.0}]},"inferText":"두었는지","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":4536.0},{"x":335.0,"y":4536.0},{"x":335.0,"y":4573.0},{"x":98.0,"y":4573.0}]},"inferText":"모르고 분실하였습니다.","inferConfidence":0.9992,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":70.0,"y":4606.0},{"x":88.0,"y":4606.0},{"x":88.0,"y":4625.0},{"x":70.0,"y":4625.0}]},"inferText":"C","inferConfidence":0.9871,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":4597.0},{"x":196.0,"y":4597.0},{"x":196.0,"y":4639.0},{"x":98.0,"y":4639.0}]},"inferText":"가족들이","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":196.0,"y":4597.0},{"x":312.0,"y":4597.0},{"x":312.0,"y":4634.0},{"x":196.0,"y":4634.0}]},"inferText":"청소하면서","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":307.0,"y":4597.0},{"x":401.0,"y":4597.0},{"x":401.0,"y":4634.0},{"x":307.0,"y":4634.0}]},"inferText":"중요하게","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":396.0,"y":4597.0},{"x":494.0,"y":4597.0},{"x":494.0,"y":4634.0},{"x":396.0,"y":4634.0}]},"inferText":"생각하지","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":489.0,"y":4597.0},{"x":657.0,"y":4597.0},{"x":657.0,"y":4634.0},{"x":489.0,"y":4634.0}]},"inferText":"않고 쓰레기통에","inferConfidence":0.9993,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":4639.0},{"x":196.0,"y":4639.0},{"x":196.0,"y":4676.0},{"x":98.0,"y":4676.0}]},"inferText":"버렸는데","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":186.0,"y":4639.0},{"x":265.0,"y":4639.0},{"x":265.0,"y":4676.0},{"x":186.0,"y":4676.0}]},"inferText":"아무리","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":261.0,"y":4639.0},{"x":335.0,"y":4639.0},{"x":335.0,"y":4676.0},{"x":261.0,"y":4676.0}]},"inferText":"뒤져도","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":331.0,"y":4639.0},{"x":433.0,"y":4639.0},{"x":433.0,"y":4676.0},{"x":331.0,"y":4676.0}]},"inferText":"없습니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":70.0,"y":4708.0},{"x":93.0,"y":4708.0},{"x":93.0,"y":4736.0},{"x":70.0,"y":4736.0}]},"inferText":"D","inferConfidence":0.9996,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":4704.0},{"x":196.0,"y":4704.0},{"x":196.0,"y":4741.0},{"x":98.0,"y":4741.0}]},"inferText":"세탁기에","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":191.0,"y":4704.0},{"x":242.0,"y":4704.0},{"x":242.0,"y":4741.0},{"x":191.0,"y":4741.0}]},"inferText":"옷과","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":238.0,"y":4704.0},{"x":345.0,"y":4704.0},{"x":345.0,"y":4741.0},{"x":238.0,"y":4741.0}]},"inferText":"함께 넣고","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":335.0,"y":4704.0},{"x":414.0,"y":4704.0},{"x":414.0,"y":4736.0},{"x":335.0,"y":4736.0}]},"inferText":"돌려서","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":405.0,"y":4704.0},{"x":480.0,"y":4704.0},{"x":480.0,"y":4736.0},{"x":405.0,"y":4736.0}]},"inferText":"티켓이","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":475.0,"y":4704.0},{"x":619.0,"y":4704.0},{"x":619.0,"y":4741.0},{"x":475.0,"y":4741.0}]},"inferText":"사라졌습니다.","inferConfidence":0.9995,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":4741.0},{"x":163.0,"y":4741.0},{"x":163.0,"y":4778.0},{"x":98.0,"y":4778.0}]},"inferText":"(혹은","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":154.0,"y":4736.0},{"x":307.0,"y":4736.0},{"x":307.0,"y":4778.0},{"x":154.0,"y":4778.0}]},"inferText":"찢어졌습니다.)","inferConfidence":0.9999,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":70.0,"y":4815.0},{"x":88.0,"y":4815.0},{"x":88.0,"y":4834.0},{"x":70.0,"y":4834.0}]},"inferText":"E","inferConfidence":0.8778,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":4811.0},{"x":149.0,"y":4811.0},{"x":149.0,"y":4839.0},{"x":98.0,"y":4839.0}]},"inferText":"집이","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":149.0,"y":4811.0},{"x":233.0,"y":4811.0},{"x":233.0,"y":4843.0},{"x":149.0,"y":4843.0}]},"inferText":"지방(or","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":224.0,"y":4811.0},{"x":326.0,"y":4811.0},{"x":326.0,"y":4848.0},{"x":224.0,"y":4848.0}]},"inferText":"해외)인데","inferConfidence":0.9993,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":321.0,"y":4811.0},{"x":428.0,"y":4811.0},{"x":428.0,"y":4848.0},{"x":321.0,"y":4848.0}]},"inferText":"깜빡 하고","inferConfidence":0.9996,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":419.0,"y":4811.0},{"x":498.0,"y":4811.0},{"x":498.0,"y":4848.0},{"x":419.0,"y":4848.0}]},"inferText":"티켓을","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":494.0,"y":4811.0},{"x":568.0,"y":4811.0},{"x":568.0,"y":4843.0},{"x":494.0,"y":4843.0}]},"inferText":"가지고","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":559.0,"y":4811.0},{"x":615.0,"y":4811.0},{"x":615.0,"y":4843.0},{"x":559.0,"y":4843.0}]},"inferText":"오지","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":610.0,"y":4811.0},{"x":685.0,"y":4811.0},{"x":685.0,"y":4843.0},{"x":610.0,"y":4843.0}]},"inferText":"않았고","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":4848.0},{"x":172.0,"y":4848.0},{"x":172.0,"y":4885.0},{"x":98.0,"y":4885.0}]},"inferText":"집에는","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":168.0,"y":4848.0},{"x":247.0,"y":4848.0},{"x":247.0,"y":4885.0},{"x":168.0,"y":4885.0}]},"inferText":"아무도","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":238.0,"y":4848.0},{"x":335.0,"y":4848.0},{"x":335.0,"y":4885.0},{"x":238.0,"y":4885.0}]},"inferText":"없습니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":331.0,"y":4848.0},{"x":396.0,"y":4848.0},{"x":396.0,"y":4885.0},{"x":331.0,"y":4885.0}]},"inferText":"(혹은","inferConfidence":0.9993,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":391.0,"y":4848.0},{"x":447.0,"y":4848.0},{"x":447.0,"y":4885.0},{"x":391.0,"y":4885.0}]},"inferText":"집에","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":442.0,"y":4848.0},{"x":494.0,"y":4848.0},{"x":494.0,"y":4885.0},{"x":442.0,"y":4885.0}]},"inferText":"있는","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":489.0,"y":4848.0},{"x":587.0,"y":4848.0},{"x":587.0,"y":4885.0},{"x":489.0,"y":4885.0}]},"inferText":"사람에게","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":582.0,"y":4848.0},{"x":661.0,"y":4848.0},{"x":661.0,"y":4885.0},{"x":582.0,"y":4885.0}]},"inferText":"사진을","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":4890.0},{"x":177.0,"y":4890.0},{"x":177.0,"y":4927.0},{"x":98.0,"y":4927.0}]},"inferText":"찍어서","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":168.0,"y":4885.0},{"x":391.0,"y":4885.0},{"x":391.0,"y":4927.0},{"x":168.0,"y":4927.0}]},"inferText":"보내라고 하겠습니다.)","inferConfidence":0.9998,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":70.0,"y":4965.0},{"x":88.0,"y":4965.0},{"x":88.0,"y":4983.0},{"x":70.0,"y":4983.0}]},"inferText":"F","inferConfidence":0.9006,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":4955.0},{"x":331.0,"y":4955.0},{"x":331.0,"y":4992.0},{"x":98.0,"y":4992.0}]},"inferText":"돌아다니다가 현장에서","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":326.0,"y":4955.0},{"x":401.0,"y":4955.0},{"x":401.0,"y":4992.0},{"x":326.0,"y":4992.0}]},"inferText":"티켓을","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":396.0,"y":4955.0},{"x":564.0,"y":4955.0},{"x":564.0,"y":4992.0},{"x":396.0,"y":4992.0}]},"inferText":"분실하였습니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":70.0,"y":5030.0},{"x":93.0,"y":5030.0},{"x":93.0,"y":5053.0},{"x":70.0,"y":5053.0}]},"inferText":"G","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":102.0,"y":5020.0},{"x":172.0,"y":5020.0},{"x":172.0,"y":5058.0},{"x":102.0,"y":5058.0}]},"inferText":"티켓을","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":168.0,"y":5020.0},{"x":242.0,"y":5020.0},{"x":242.0,"y":5058.0},{"x":168.0,"y":5058.0}]},"inferText":"가지고","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":238.0,"y":5020.0},{"x":293.0,"y":5020.0},{"x":293.0,"y":5058.0},{"x":238.0,"y":5058.0}]},"inferText":"있는","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":289.0,"y":5020.0},{"x":363.0,"y":5020.0},{"x":363.0,"y":5058.0},{"x":289.0,"y":5058.0}]},"inferText":"사람이","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":359.0,"y":5020.0},{"x":433.0,"y":5020.0},{"x":433.0,"y":5058.0},{"x":359.0,"y":5058.0}]},"inferText":"공연장","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":428.0,"y":5020.0},{"x":484.0,"y":5020.0},{"x":484.0,"y":5058.0},{"x":428.0,"y":5058.0}]},"inferText":"안에","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":480.0,"y":5020.0},{"x":605.0,"y":5020.0},{"x":605.0,"y":5058.0},{"x":480.0,"y":5058.0}]},"inferText":"들어가 있고","inferConfidence":0.9999,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":5062.0},{"x":224.0,"y":5062.0},{"x":224.0,"y":5100.0},{"x":98.0,"y":5100.0}]},"inferText":"연락이 되지","inferConfidence":0.9996,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":219.0,"y":5062.0},{"x":317.0,"y":5062.0},{"x":317.0,"y":5100.0},{"x":219.0,"y":5100.0}]},"inferText":"않습니다","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":70.0,"y":5132.0},{"x":88.0,"y":5132.0},{"x":88.0,"y":5151.0},{"x":70.0,"y":5151.0}]},"inferText":"H","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":5123.0},{"x":242.0,"y":5123.0},{"x":242.0,"y":5165.0},{"x":98.0,"y":5165.0}]},"inferText":"친구가 티켓을","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":238.0,"y":5123.0},{"x":387.0,"y":5123.0},{"x":387.0,"y":5165.0},{"x":238.0,"y":5165.0}]},"inferText":"가지고 있는데","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":377.0,"y":5123.0},{"x":433.0,"y":5123.0},{"x":433.0,"y":5160.0},{"x":377.0,"y":5160.0}]},"inferText":"공연","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":424.0,"y":5123.0},{"x":480.0,"y":5123.0},{"x":480.0,"y":5155.0},{"x":424.0,"y":5155.0}]},"inferText":"시작","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":470.0,"y":5118.0},{"x":717.0,"y":5118.0},{"x":717.0,"y":5165.0},{"x":470.0,"y":5165.0}]},"inferText":"후에 도착한다고 합니다.","inferConfidence":0.9996,"type":"NORMAL","lineBreak":true}]}],"originalFileName":"25006132-03","imageType":"jpg"} """

In [6]:
paras = parse_paragraphs(data)

for p in paras:
    print(p["top_left"], p["text"])

TypeError: string indices must be integers

에러 해결 도전

In [1]:
import json

def parse_paragraphs(raw, paragraph_gap=20):
    # -- 1) raw가 문자열이라면 dict로 변환
    data = json.loads(raw) if isinstance(raw, str) else raw

    fields = data["fields"]
    # (이후는 이전과 동일)
    fields.sort(key=lambda f: f["boundingPoly"]["vertices"][0]["y"])
    
    lines = []
    cur_line = []
    for f in fields:
        text = f["inferText"]
        pt = f["boundingPoly"]["vertices"][0]
        cur_line.append((text, pt))
        if f.get("lineBreak", False):
            lines.append(cur_line)
            cur_line = []
    if cur_line:
        lines.append(cur_line)
    
    paragraphs = []
    cur_para = []
    prev_y = None
    for line in lines:
        line_y = min(p["y"] for _, p in line)
        if prev_y is not None and line_y - prev_y > paragraph_gap:
            paragraphs.append(cur_para)
            cur_para = []
        cur_para.append(line)
        prev_y = line_y
    if cur_para:
        paragraphs.append(cur_para)
    
    result = []
    for para in paragraphs:
        texts = [" ".join(w for w, _ in line) for line in para]
        full_text = "\n".join(texts)
        all_pts = [pt for line in para for _, pt in line]
        top_left = {"x": min(p["x"] for p in all_pts), "y": min(p["y"] for p in all_pts)}
        result.append({"text": full_text, "top_left": top_left})
    
    return result


In [ ]:

for i, p in enumerate(paras, 1):
    print(f"[단락 {i}] {p['top_left']}\n{p['text']}\n")

3번째 시도도

In [ ]:
import json

def parse_paragraphs(raw, paragraph_gap=20):
    # 1) raw가 문자열이면 dict로 변환
    data = json.loads(raw) if isinstance(raw, str) else raw

    # 2) fields 추출: images 배열 안의 첫 번째 객체에서 가져옴
    #    (여러 이미지가 있다면 for문으로 돌려도 되고, 단 하나만 처리할 땐 [0] 사용)
    img = data.get("images", [])[0]
    if img is None or "fields" not in img:
        raise ValueError("`images[0]['fields']`를 찾을 수 없습니다. JSON 구조를 확인해주세요.")
    fields = img["fields"]

    # 3) y 기준 정렬
    fields.sort(key=lambda f: f["boundingPoly"]["vertices"][0]["y"])

    # 4) lineBreak로 줄 묶기
    lines = []
    cur_line = []
    for f in fields:
        text = f["inferText"]
        pt = f["boundingPoly"]["vertices"][0]
        cur_line.append((text, pt))
        if f.get("lineBreak", False):
            lines.append(cur_line)
            cur_line = []
    if cur_line:
        lines.append(cur_line)

    # 5) 줄 간격으로 단락 묶기
    paragraphs = []
    cur_para = []
    prev_y = None
    for line in lines:
        line_y = min(p["y"] for _, p in line)
        if prev_y is not None and line_y - prev_y > paragraph_gap:
            paragraphs.append(cur_para)
            cur_para = []
        cur_para.append(line)
        prev_y = line_y
    if cur_para:
        paragraphs.append(cur_para)

    # 6) 결과 포맷
    result = []
    for para in paragraphs:
        texts = [" ".join(w for w, _ in line) for line in para]
        full_text = "\n".join(texts)
        all_pts = [pt for line in para for _, pt in line]
        top_left = {"x": min(p["x"] for p in all_pts), "y": min(p["y"] for p in all_pts)}
        result.append({"text": full_text, "top_left": top_left})

    return result


# -- 사용 예시 (ipynb 셀에 붙여넣고 실행) --

# 1) 복붙한 JSON 문자열을 raw_json 변수에 담고


# 2) parse_paragraphs 호출
paras = parse_paragraphs(raw_json)

# 3) 결과 출력
for i, p in enumerate(paras, 1):
    print(f"[단락 {i}] 좌표={p['top_left']}\n{p['text']}\n")


[단락 1] 좌표={'x': 126.0, 'y': 47.0}
티켓 배송 현장 안내
및 수령 티켓 배송 안내

[단락 2] 좌표={'x': 61.0, 'y': 261.0}
공연당일 혼잡을 막기 위해 예매 초기에는 일괄 배송 방식으로

[단락 3] 좌표={'x': 61.0, 'y': 293.0}
9일(월)부터 순차 운영합니다. (일괄 배송일: 6월 배송)

[단락 4] 좌표={'x': 61.0, 'y': 368.0}
일괄 배송 티켓은 5월 28일(수) 10:00(KST)까지 예매 완료된

[단락 5] 좌표={'x': 61.0, 'y': 405.0}
건부터 배송될 예정이며, 일괄 배송일 이후
순차적으로 예매하시는 분들은 현장 수령만 가능합니다.

[단락 6] 좌표={'x': 61.0, 'y': 507.0}
12:00(KST)까지 배송지 변경은 9일(월) 낮 가능합니다.
6월 티켓 구매 전 수령 정보(배송 주소, 수령인, 연락처)가 정확한지

[단락 7] 좌표={'x': 61.0, 'y': 587.0}
한번 확인하여 .다. 배송 수령 정보의
다시 주시기 바랍 후에는 변경이 어려우며, 반송 재배송은불가합니다. (현장 수령만 가능)

[단락 8] 좌표={'x': 284.0, 'y': 633.0}
후 않으시길

[단락 9] 좌표={'x': 61.0, 'y': 694.0}
티켓 수령 온라인상에 티켓의 올리지 후, 이미지를 절대 통한 바라며, 불법 복제를 가짜 티켓으로 위조될 소지가 있으니

[단락 10] 좌표={'x': 61.0, 'y': 767.0}
유의하시어 피해를 입지 않으시길 바랍니다.
이 점 판매하는 위조 티켓을 위법행위로 추정되는 경우 주최, 주관 측의

[단락 11] 좌표={'x': 61.0, 'y': 843.0}
등 권한으로 사전통보 없이 당사자의 티켓은 무효처리 되며,

[단락 12] 좌표={'x': 298.0, 'y': 880.0}
거래 처리됨을

[단락 13] 좌표={'x': 61.0, 'y': 917.0}
공연당일 현장에 티켓을 소지하였을지라도 입장 불

## top y 좌표만 나오게 하기

In [ ]:
import json

def parse_paragraphs(raw, paragraph_gap=20):
    # 문자열일 경우 dict로 변환
    data = json.loads(raw) if isinstance(raw, str) else raw

    # images[0] 안의 fields
    img = data.get("images", [])[0]
    if not img or "fields" not in img:
        raise ValueError("`images[0]['fields']`를 찾을 수 없습니다.")
    fields = img["fields"]

    # y 기준 정렬
    fields.sort(key=lambda f: f["boundingPoly"]["vertices"][0]["y"])

    # lineBreak로 줄 묶기
    lines, cur_line = [], []
    for f in fields:
        text = f["inferText"]
        pt = f["boundingPoly"]["vertices"][0]
        cur_line.append((text, pt))
        if f.get("lineBreak", False):
            lines.append(cur_line)
            cur_line = []
    if cur_line:
        lines.append(cur_line)

    # 줄 간격으로 단락 묶기
    paragraphs, cur_para, prev_y = [], [], None
    for line in lines:
        line_y = min(p["y"] for _, p in line)
        if prev_y is not None and line_y - prev_y > paragraph_gap:
            paragraphs.append(cur_para)
            cur_para = []
        cur_para.append(line)
        prev_y = line_y
    if cur_para:
        paragraphs.append(cur_para)

    # 결과: text + top_y
    result = []
    for para in paragraphs:
        texts = [" ".join(w for w, _ in line) for line in para]
        full_text = "\n".join(texts)
        all_ys = [pt["y"] for line in para for _, pt in line]
        top_y = min(all_ys)
        result.append({"text": full_text, "top_y": top_y})

    return result

raw_json = """ces":[{"x":331.0,"y":4848.0},{"x":396.0,"y":4848.0},{"x":396.0,"y":4885.0},{"x":331.0,"y":4885.0}]},"inferText":"(혹은","inferConfidence":0.9993,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":391.0,"y":4848.0},{"x":447.0,"y":4848.0},{"x":447.0,"y":4885.0},{"x":391.0,"y":4885.0}]},"inferText":"집에","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":442.0,"y":4848.0},{"x":494.0,"y":4848.0},{"x":494.0,"y":4885.0},{"x":442.0,"y":4885.0}]},"inferText":"있는","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":489.0,"y":4848.0},{"x":587.0,"y":4848.0},{"x":587.0,"y":4885.0},{"x":489.0,"y":4885.0}]},"inferText":"사람에게","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":582.0,"y":4848.0},{"x":661.0,"y":4848.0},{"x":661.0,"y":4885.0},{"x":582.0,"y":4885.0}]},"inferText":"사진을","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":4890.0},{"x":177.0,"y":4890.0},{"x":177.0,"y":4927.0},{"x":98.0,"y":4927.0}]},"inferText":"찍어서","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":168.0,"y":4885.0},{"x":391.0,"y":4885.0},{"x":391.0,"y":4927.0},{"x":168.0,"y":4927.0}]},"inferText":"보내라고 하겠습니다.)","inferConfidence":0.9998,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":70.0,"y":4965.0},{"x":88.0,"y":4965.0},{"x":88.0,"y":4983.0},{"x":70.0,"y":4983.0}]},"inferText":"F","inferConfidence":0.9006,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":4955.0},{"x":331.0,"y":4955.0},{"x":331.0,"y":4992.0},{"x":98.0,"y":4992.0}]},"inferText":"돌아다니다가 현장에서","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":326.0,"y":4955.0},{"x":401.0,"y":4955.0},{"x":401.0,"y":4992.0},{"x":326.0,"y":4992.0}]},"inferText":"티켓을","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":396.0,"y":4955.0},{"x":564.0,"y":4955.0},{"x":564.0,"y":4992.0},{"x":396.0,"y":4992.0}]},"inferText":"분실하였습니다.","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":70.0,"y":5030.0},{"x":93.0,"y":5030.0},{"x":93.0,"y":5053.0},{"x":70.0,"y":5053.0}]},"inferText":"G","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":102.0,"y":5020.0},{"x":172.0,"y":5020.0},{"x":172.0,"y":5058.0},{"x":102.0,"y":5058.0}]},"inferText":"티켓을","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":168.0,"y":5020.0},{"x":242.0,"y":5020.0},{"x":242.0,"y":5058.0},{"x":168.0,"y":5058.0}]},"inferText":"가지고","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":238.0,"y":5020.0},{"x":293.0,"y":5020.0},{"x":293.0,"y":5058.0},{"x":238.0,"y":5058.0}]},"inferText":"있는","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":289.0,"y":5020.0},{"x":363.0,"y":5020.0},{"x":363.0,"y":5058.0},{"x":289.0,"y":5058.0}]},"inferText":"사람이","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":359.0,"y":5020.0},{"x":433.0,"y":5020.0},{"x":433.0,"y":5058.0},{"x":359.0,"y":5058.0}]},"inferText":"공연장","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":428.0,"y":5020.0},{"x":484.0,"y":5020.0},{"x":484.0,"y":5058.0},{"x":428.0,"y":5058.0}]},"inferText":"안에","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":480.0,"y":5020.0},{"x":605.0,"y":5020.0},{"x":605.0,"y":5058.0},{"x":480.0,"y":5058.0}]},"inferText":"들어가 있고","inferConfidence":0.9999,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":5062.0},{"x":224.0,"y":5062.0},{"x":224.0,"y":5100.0},{"x":98.0,"y":5100.0}]},"inferText":"연락이 되지","inferConfidence":0.9996,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":219.0,"y":5062.0},{"x":317.0,"y":5062.0},{"x":317.0,"y":5100.0},{"x":219.0,"y":5100.0}]},"inferText":"않습니다","inferConfidence":1.0,"type":"NORMAL","lineBreak":true},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":70.0,"y":5132.0},{"x":88.0,"y":5132.0},{"x":88.0,"y":5151.0},{"x":70.0,"y":5151.0}]},"inferText":"H","inferConfidence":0.9998,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":98.0,"y":5123.0},{"x":242.0,"y":5123.0},{"x":242.0,"y":5165.0},{"x":98.0,"y":5165.0}]},"inferText":"친구가 티켓을","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":238.0,"y":5123.0},{"x":387.0,"y":5123.0},{"x":387.0,"y":5165.0},{"x":238.0,"y":5165.0}]},"inferText":"가지고 있는데","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":377.0,"y":5123.0},{"x":433.0,"y":5123.0},{"x":433.0,"y":5160.0},{"x":377.0,"y":5160.0}]},"inferText":"공연","inferConfidence":0.9999,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":424.0,"y":5123.0},{"x":480.0,"y":5123.0},{"x":480.0,"y":5155.0},{"x":424.0,"y":5155.0}]},"inferText":"시작","inferConfidence":1.0,"type":"NORMAL","lineBreak":false},{"valueType":"ALL","boundingPoly":{"vertices":[{"x":470.0,"y":5118.0},{"x":717.0,"y":5118.0},{"x":717.0,"y":5165.0},{"x":470.0,"y":5165.0}]},"inferText":"후에 도착한다고 합니다.","inferConfidence":0.9996,"type":"NORMAL","lineBreak":true}]}],"originalFileName":"25006132-03","imageType":"jpg"}"""
paras = parse_paragraphs(raw_json)
for p in paras:
    print(f"[y={p['top_y']}] {p['text']}\n")

[y=47.0] 티켓 배송 현장 안내
및 수령 티켓 배송 안내

[y=261.0] 공연당일 혼잡을 막기 위해 예매 초기에는 일괄 배송 방식으로

[y=293.0] 9일(월)부터 순차 운영합니다. (일괄 배송일: 6월 배송)

[y=368.0] 일괄 배송 티켓은 5월 28일(수) 10:00(KST)까지 예매 완료된

[y=405.0] 건부터 배송될 예정이며, 일괄 배송일 이후
순차적으로 예매하시는 분들은 현장 수령만 가능합니다.

[y=507.0] 12:00(KST)까지 배송지 변경은 9일(월) 낮 가능합니다.
6월 티켓 구매 전 수령 정보(배송 주소, 수령인, 연락처)가 정확한지

[y=587.0] 한번 확인하여 .다. 배송 수령 정보의
다시 주시기 바랍 후에는 변경이 어려우며, 반송 재배송은불가합니다. (현장 수령만 가능)

[y=633.0] 후 않으시길

[y=694.0] 티켓 수령 온라인상에 티켓의 올리지 후, 이미지를 절대 통한 바라며, 불법 복제를 가짜 티켓으로 위조될 소지가 있으니

[y=767.0] 유의하시어 피해를 입지 않으시길 바랍니다.
이 점 판매하는 위조 티켓을 위법행위로 추정되는 경우 주최, 주관 측의

[y=843.0] 등 권한으로 사전통보 없이 당사자의 티켓은 무효처리 되며,

[y=880.0] 거래 처리됨을

[y=917.0] 공연당일 현장에 티켓을 소지하였을지라도 입장 불가 반드시 상기해 주시기 바랍니다.

[y=1020.0] 고객의 부재로 인해 반송된 티켓은 현장 매표소에서 수령하여야

[y=1062.0] 합니다. 현장 수령 증빙서류를 지참하셔야 티켓 수령이
시 가능하오니 아래의 현장 수령에 대한 안내사항을 확인 부탁드립니다.

[y=1164.0] 공연일 일주일 전까지도 배송된 티켓을 수령하지 못했을 시

[y=1198.0] 1555)로 티켓 문의 NOL 고객센터(1544- 바랍니다.

[y=1271.0] 글로벌 페이지에서 예매한 티켓은 현장 수령만 가능합니다.

[y=1369.0] 현장 티켓 수령 안내

[y=1451.0] 시작

In [2]:
import json
from pprint import pprint

def parse_paragraphs(raw, paragraph_gap=20):
    # 1) raw가 문자열이면 dict로 변환
    data = json.loads(raw) if isinstance(raw, str) else raw

    # 2) fields 추출
    img = data.get("images", [None])[0]
    if not img or "fields" not in img:
        raise ValueError("`images[0]['fields']`를 찾을 수 없습니다.")
    fields = img["fields"]

    # 3) y 기준 정렬
    fields.sort(key=lambda f: f["boundingPoly"]["vertices"][0]["y"])

    # 4) 줄 묶기 (lineBreak 기준)
    lines, cur_line = [], []
    for f in fields:
        text = f["inferText"]
        pt = f["boundingPoly"]["vertices"][0]
        cur_line.append((text, pt))
        if f.get("lineBreak", False):
            lines.append(cur_line)
            cur_line = []
    if cur_line:
        lines.append(cur_line)

    # 5) 단락 묶기 (줄 간격 기준)
    paragraphs, cur_para, prev_y = [], [], None
    for line in lines:
        line_y = min(pt["y"] for _, pt in line)
        if prev_y is not None and line_y - prev_y > paragraph_gap:
            paragraphs.append(cur_para)
            cur_para = []
        cur_para.append(line)
        prev_y = line_y
    if cur_para:
        paragraphs.append(cur_para)

    # 6) 결과 생성: text + top_y + bottom_y
    result = []
    for para in paragraphs:
        # 문단 텍스트
        texts = [" ".join(w for w, _ in line) for line in para]
        full_text = "\n".join(texts)
        # y 값만 모아서
        all_ys = [pt["y"] for line in para for _, pt in line]
        result.append({
            "text": full_text,
            "top_y":    min(all_ys),
            "bottom_y": max(all_ys)
        })

    return result
paras = parse_paragraphs(raw_json)

for p in paras:
    print(f"[top={p['top_y']}, bottom={p['bottom_y']}]")
    print(p["text"])
    print()



[top=47.0, bottom=172.0]
티켓 배송 현장 안내
및 수령 티켓 배송 안내

[top=261.0, bottom=261.0]
공연당일 혼잡을 막기 위해 예매 초기에는 일괄 배송 방식으로

[top=293.0, bottom=298.0]
9일(월)부터 순차 운영합니다. (일괄 배송일: 6월 배송)

[top=368.0, bottom=368.0]
일괄 배송 티켓은 5월 28일(수) 10:00(KST)까지 예매 완료된

[top=405.0, bottom=447.0]
건부터 배송될 예정이며, 일괄 배송일 이후
순차적으로 예매하시는 분들은 현장 수령만 가능합니다.

[top=507.0, bottom=550.0]
12:00(KST)까지 배송지 변경은 9일(월) 낮 가능합니다.
6월 티켓 구매 전 수령 정보(배송 주소, 수령인, 연락처)가 정확한지

[top=587.0, bottom=629.0]
한번 확인하여 .다. 배송 수령 정보의
다시 주시기 바랍 후에는 변경이 어려우며, 반송 재배송은불가합니다. (현장 수령만 가능)

[top=633.0, bottom=689.0]
후 않으시길

[top=694.0, bottom=736.0]
티켓 수령 온라인상에 티켓의 올리지 후, 이미지를 절대 통한 바라며, 불법 복제를 가짜 티켓으로 위조될 소지가 있으니

[top=767.0, bottom=838.0]
유의하시어 피해를 입지 않으시길 바랍니다.
이 점 판매하는 위조 티켓을 위법행위로 추정되는 경우 주최, 주관 측의

[top=843.0, bottom=876.0]
등 권한으로 사전통보 없이 당사자의 티켓은 무효처리 되며,

[top=880.0, bottom=913.0]
거래 처리됨을

[top=917.0, bottom=959.0]
공연당일 현장에 티켓을 소지하였을지라도 입장 불가 반드시 상기해 주시기 바랍니다.

[top=1020.0, bottom=1020.0]
고객의 부재로 인해 반송된 티켓은 현장 매표소에서 수령하여야

[top=1062.0, bottom=1099.0]